## training SVM, LDA, RF

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from mne.decoding import CSP

def detect_contralateral_pattern(trial_data, label, channels=['C3', 'C4'], fs=250):
    """
    Detect contralateral ERD pattern in motor imagery data
    Returns True if the expected pattern is present, False otherwise
    """
    # Bandpass filter (alpha/beta rhythms)
    b, a = signal.butter(4, [8, 30], btype='bandpass', fs=fs)
    filtered_data = signal.filtfilt(b, a, trial_data[channels], axis=0)
    
    # Calculate band power (variance)
    c3_power = np.var(filtered_data[:, 0])
    c4_power = np.var(filtered_data[:, 1])
    
    # Check for expected contralateral pattern
    if label == 'Left':
        return c4_power < c3_power * 0.9  # At least 10% decrease contralaterally
    elif label == 'Right':
        return c3_power < c4_power * 0.9
    return False

def extract_time_window(trial_data, start_time, end_time, fs=250):
    """Extract specific time window from trial data"""
    start_idx = int(start_time * fs)
    end_idx = int(end_time * fs)
    return trial_data.iloc[start_idx:end_idx]

# Configuration
WINDOW = (5.75, 6.75)  # Target time window
FS = 250  # Sampling rate
CHANNELS = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']  # EEG channels

# Load data index files
base_path = '/kaggle/input/mtcaic3/'
train_df = pd.read_csv(base_path + 'train.csv')
valid_df = pd.read_csv(base_path + 'validation.csv')

# Filter for MI tasks only
train_mi = train_df[train_df['task'] == 'MI'].copy()
valid_mi = valid_df[valid_df['task'] == 'MI'].copy()

# Function to load EEG data (from competition instructions)
def load_trial_data(row):
    id_num = row['id']
    if id_num <= 4800:
        dataset = 'train'
    elif id_num <= 4900:
        dataset = 'validation'
    else:
        dataset = 'test'

    # Construct the path to EEGdata.csv
    eeg_path = f"{base_path}/{row['task']}/{dataset}/{row['subject_id']}/{row['trial_session']}/EEGdata.csv"

    # Load the entire EEG file
    eeg_data = pd.read_csv(eeg_path)

    # Calculate indices for the specific trial
    trial_num = int(row['trial'])
    if row['task'] == 'MI':
        samples_per_trial = 2250  # 9 seconds * 250 Hz
    else:  # SSVEP
        samples_per_trial = 1750  # 7 seconds * 250 Hz

    start_idx = (trial_num - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial - 1

    # Extract the trial data
    trial_data = eeg_data.iloc[start_idx:end_idx+1]
    return trial_data

# 1. Filter trials with contralateral pattern in target window
def filter_pattern_trials(df, window):
    pattern_mask = []
    for _, row in df.iterrows():
        full_trial = load_trial_data(row)
        window_data = extract_time_window(full_trial, *window)
        if detect_contralateral_pattern(window_data[CHANNELS], row['label']):
            pattern_mask.append(True)
        else:
            pattern_mask.append(False)
    return df[pattern_mask]

train_filtered = filter_pattern_trials(train_mi, WINDOW)
valid_filtered = filter_pattern_trials(valid_mi, WINDOW)

print(f"Original MI trials: Train={len(train_mi)}, Valid={len(valid_mi)}")
print(f"Trials with contralateral pattern: Train={len(train_filtered)}, Valid={len(valid_filtered)}")

# 2. Prepare data for modeling
def prepare_data(df, window):
    X, y = [], []
    for _, row in df.iterrows():
        full_trial = load_trial_data(row)
        window_data = extract_time_window(full_trial, *window)
        X.append(window_data[CHANNELS].values.T)  # Shape: (channels, time)
        y.append(row['label'])
    return np.array(X), np.array(y)

X_train, y_train = prepare_data(train_filtered, WINDOW)
X_valid, y_valid = prepare_data(valid_filtered, WINDOW)

# 3. Train and evaluate models
models = {
    "CSP+LDA": make_pipeline(CSP(n_components=4), LinearDiscriminantAnalysis()),
    "CSP+SVM": make_pipeline(CSP(n_components=4), SVC(kernel='linear', probability=True)),
    "CSP+RF": make_pipeline(CSP(n_components=4), RandomForestClassifier(n_estimators=100))
}

for name, model in models.items():
    print(f"\n=== Training {name} ===")
    
    # Train model
    model.fit(X_train, y_train)
    
    # Get predictions and confidences
    if hasattr(model, 'predict_proba'):
        probas = model.predict_proba(X_valid)
        confidences = np.max(probas, axis=1)
        predictions = model.classes_[np.argmax(probas, axis=1)]
    else:  # For models without predict_proba
        predictions = model.predict(X_valid)
        decision = model.decision_function(X_valid)
        confidences = 1 / (1 + np.exp(-np.abs(decision)))
    
    # Print classification report
    print(classification_report(y_valid, predictions))
    
    # Print predictions with confidence
    results = pd.DataFrame({
        'True Label': y_valid,
        'Predicted': predictions,
        'Confidence': confidences
    })
    print(f"\nValidation set predictions with confidence ({name}):")
    print(results.head(10))
    print("...\n")

print("Analysis complete. Focused on 5.75-6.75s window with contralateral pattern trials.")

## EEGNET

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F # Import functional for ELU
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from mne.decoding import CSP
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
import time
import matplotlib.pyplot as plt
from tqdm import tqdm
import copy # For deep copying model weights

# EEGNet Architecture (PyTorch Implementation) - MODIFIED
class EEGNet(nn.Module):
    def __init__(self, num_channels=4, num_classes=2, sampling_rate=250, dropout_rate=0.5): # Increased dropout
        super(EEGNet, self).__init__()
        self.T = int((6.75 - 5.75) * sampling_rate)
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, sampling_rate//2), padding=(0, sampling_rate//4), bias=False)
        self.batchnorm1 = nn.BatchNorm2d(16)
        
        # Layer 2
        self.conv2 = nn.Conv2d(16, 32, (num_channels, 1), groups=16, bias=False)
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.pooling2 = nn.AvgPool2d((1, 4))
        self.dropout2 = nn.Dropout(dropout_rate) # Using parameter
        
        # Layer 3
        self.conv3 = nn.Conv2d(32, 32, (1, 16), padding=(0, 8), groups=32, bias=False)
        self.conv3_1 = nn.Conv2d(32, 32, 1, bias=False)
        self.batchnorm3 = nn.BatchNorm2d(32)
        self.pooling3 = nn.AvgPool2d((1, 8))
        self.dropout3 = nn.Dropout(dropout_rate) # Using parameter
        
        # FC Layer - Made robust with a helper function
        fc_input_size = self._get_fc_input_size((1, num_channels, self.T))
        self.fc = nn.Linear(fc_input_size, num_classes)
        
    def _get_fc_input_size(self, shape):
        # Helper to dynamically calculate the flattened size for the FC layer
        with torch.no_grad():
            x = torch.rand(1, *shape)
            x = self.forward_features(x)
            return x.view(1, -1).size(1)

    def forward_features(self, x):
        # Feature extraction part of the forward pass
        x = F.elu(self.batchnorm1(self.conv1(x))) # Using ELU activation
        x = F.elu(self.batchnorm2(self.conv2(x)))
        x = self.pooling2(x)
        x = self.dropout2(x)
        
        x = F.elu(self.batchnorm3(self.conv3_1(self.conv3(x))))
        x = self.pooling3(x)
        x = self.dropout3(x)
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# --- 1. OVERFITTING FIX: Data Augmentation ---
def augment_data(data, noise_level=0.01):
    """Adds Gaussian noise to the data."""
    noise = torch.randn_like(data) * noise_level
    return data + noise

# --- Assume X_train, y_train, X_valid, y_valid are loaded and prepared ---
# Example placeholder data if you run this standalone
# X_train = np.random.randn(1121, 8, 250)
# y_train = np.random.choice(['Left', 'Right'], 1121, p=[0.7, 0.3]) # Imbalanced example
# X_valid = np.random.randn(24, 8, 250)
# y_valid = np.random.choice(['Left', 'Right'], 24)

# Apply CSP with TIME-SERIES output
csp = CSP(n_components=4, transform_into='csp_space')
X_train_csp = csp.fit_transform(X_train, y_train)
X_valid_csp = csp.transform(X_valid)

# Reshape data for EEGNet
X_train_dl = X_train_csp[:, np.newaxis, :, :]
X_valid_dl = X_valid_csp[:, np.newaxis, :, :]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Normalize each channel independently
# A better approach is to fit on train set and transform valid set to avoid data leakage
# But we will keep your original method for now to focus on the key changes
def normalize_channels(data):
    scaler = StandardScaler()
    normalized = np.zeros_like(data)
    for i in range(data.shape[0]):
        for j in range(data.shape[2]):
            channel_data = data[i, 0, j, :]
            normalized[i, 0, j, :] = scaler.fit_transform(channel_data.reshape(-1, 1)).flatten()
    return normalized

X_train_norm = normalize_channels(X_train_dl)
X_valid_norm = normalize_channels(X_valid_dl)

# Convert to PyTorch tensors
train_tensor = torch.tensor(X_train_norm, dtype=torch.float32)
valid_tensor = torch.tensor(X_valid_norm, dtype=torch.float32)
train_labels_numerical = np.array([1 if label == 'Right' else 0 for label in y_train])
valid_labels_numerical = np.array([1 if label == 'Right' else 0 for label in y_valid])

train_labels = torch.tensor(train_labels_numerical, dtype=torch.long)
valid_labels = torch.tensor(valid_labels_numerical, dtype=torch.long)

# Create DataLoaders
train_dataset = TensorDataset(train_tensor, train_labels)
valid_dataset = TensorDataset(valid_tensor, valid_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32)

# --- 2. CLASS IMBALANCE FIX: Calculate class weights ---
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels_numerical), y=train_labels_numerical)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)
print(f"Calculated class weights: {class_weights_tensor.cpu().numpy()}")

# Initialize model, loss, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EEGNet(num_channels=4, num_classes=2, sampling_rate=250, dropout_rate=0.5).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor) # Use weighted loss
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4) # Slightly increased weight decay
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True)

# Training loop with Early Stopping
def train_model(model, train_loader, valid_loader, epochs=100):
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'lr': [], 'epoch_time': []}
    
    # --- 3. OVERFITTING FIX: Early Stopping parameters ---
    best_val_loss = float('inf')
    patience = 20  # Stop after 10 epochs of no improvement
    patience_counter = 0
    best_model_wts = None

    for epoch in range(epochs):
        epoch_start = time.time()
        
        # Training phase
        model.train()
        running_loss, correct_train, total_train = 0.0, 0, 0
        train_iter = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Train]', leave=False)
        for inputs, labels in train_iter:
            # Apply augmentation
            inputs = augment_data(inputs, noise_level=0.02).to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
            train_iter.set_postfix({'loss': running_loss/total_train, 'acc': correct_train/total_train})

        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = correct_train / total_train
        history['train_loss'].append(epoch_train_loss)
        history['train_acc'].append(epoch_train_acc)
        
        # Validation phase
        model.eval()
        val_loss, correct_val, total_val = 0.0, 0, 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            val_iter = tqdm(valid_loader, desc=f'Epoch {epoch+1}/{epochs} [Valid]', leave=False)
            for inputs, labels in val_iter:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                val_iter.set_postfix({'loss': val_loss/total_val, 'acc': correct_val/total_val})

        epoch_val_loss = val_loss / len(valid_loader.dataset)
        epoch_val_acc = correct_val / total_val
        history['val_loss'].append(epoch_val_loss)
        history['val_acc'].append(epoch_val_acc)
        
        current_lr = optimizer.param_groups[0]['lr']
        history['lr'].append(current_lr)
        scheduler.step(epoch_val_acc)
        epoch_time = time.time() - epoch_start
        history['epoch_time'].append(epoch_time)
        
        print(f'\nEpoch {epoch+1}/{epochs} Summary:')
        print(f'  Time: {epoch_time:.2f}s | LR: {current_lr:.6f}')
        print(f'  Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.4f}')
        print(f'  Val Loss:   {epoch_val_loss:.4f} | Val Acc:   {epoch_val_acc:.4f}')
        
        # Early Stopping Logic
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            patience_counter = 0
            best_model_wts = copy.deepcopy(model.state_dict())
            print(f"  Validation loss improved. Saving best model.")
        else:
            patience_counter += 1
            print(f"  Validation loss did not improve. Patience: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print("\nEarly stopping triggered.")
            break
            
    # Load best model weights
    if best_model_wts:
        model.load_state_dict(best_model_wts)
        
    return all_preds, all_labels, history

# ... (rest of the script for reporting and plotting remains the same) ...
# Train the model
print("Training EEGNet model...")
preds, true_labels, history = train_model(model, train_loader, valid_loader, epochs=30)

# Convert numerical labels back to string
label_map = {0: 'Left', 1: 'Right'}
str_preds = [label_map[p] for p in preds]
str_true = [label_map[l] for l in true_labels]

# Print final classification report
print("\nFinal Classification Report:")
print(classification_report(str_true, str_preds))

# Print predictions with confidence
model.eval()
confidences = []
with torch.no_grad():
    for inputs, _ in valid_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        probs = torch.softmax(outputs, dim=1)
        conf, _ = torch.max(probs, dim=1)
        confidences.extend(conf.cpu().numpy())

results = pd.DataFrame({
    'True Label': str_true,
    'Predicted': str_preds,
    'Confidence': confidences
})
print("\nValidation set predictions with confidence (CSP + EEGNet):")
print(results)

# Save model
torch.save(model.state_dict(), 'csp_eegnet_model.pth')
print("Model saved to 'csp_eegnet_model.pth'")

# Print training summary
print("\nTraining Summary:")
print(f"Best Validation Accuracy: {max(history['val_acc']):.4f}")
print(f"Total Training Time: {sum(history['epoch_time']):.2f} seconds")
print(f"Average Epoch Time: {np.mean(history['epoch_time']):.2f} seconds")
print(f"Final Learning Rate: {history['lr'][-1]:.6f}")

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from mne.decoding import CSP
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
import time
import matplotlib.pyplot as plt
from tqdm import tqdm
import copy

# EEGNet Architecture (PyTorch Implementation) - MODIFIED
class EEGNet(nn.Module):
    def __init__(self, num_channels=4, num_classes=2, sampling_rate=250, dropout_rate=0.6): # Increased dropout
        super(EEGNet, self).__init__()
        self.T = int((6.75 - 5.75) * sampling_rate)

        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, sampling_rate // 2), padding=(0, sampling_rate // 4), bias=False)
        self.batchnorm1 = nn.BatchNorm2d(16)

        # Layer 2
        self.conv2 = nn.Conv2d(16, 32, (num_channels, 1), groups=16, bias=False)
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.pooling2 = nn.AvgPool2d((1, 4))
        self.dropout2 = nn.Dropout(dropout_rate)

        # Layer 3
        self.conv3 = nn.Conv2d(32, 32, (1, 16), padding=(0, 8), groups=32, bias=False)
        self.conv3_1 = nn.Conv2d(32, 32, 1, bias=False)
        self.batchnorm3 = nn.BatchNorm2d(32)
        self.pooling3 = nn.AvgPool2d((1, 8))
        self.dropout3 = nn.Dropout(dropout_rate)

        # FC Layer
        fc_input_size = self._get_fc_input_size((1, num_channels, self.T))
        self.fc = nn.Linear(fc_input_size, num_classes)
        self.fc_dropout = nn.Dropout(0.7) # Higher dropout for the final classifier

    def _get_fc_input_size(self, shape):
        with torch.no_grad():
            x = torch.rand(1, *shape)
            x = self.forward_features(x)
            return x.view(1, -1).size(1)

    def forward_features(self, x):
        x = F.elu(self.batchnorm1(self.conv1(x)))
        x = F.elu(self.batchnorm2(self.conv2(x)))
        x = self.pooling2(x)
        x = self.dropout2(x)

        x = F.elu(self.batchnorm3(self.conv3_1(self.conv3(x))))
        x = self.pooling3(x)
        x = self.dropout3(x)
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = x.view(x.size(0), -1)
        x = self.fc_dropout(x)
        x = self.fc(x)
        return x

# --- 1. CLASS-WISE Data Augmentation ---
def augment_data_classwise(data, labels, minority_class=1, noise_level=0.02, shift_max=10):
    """
    Applies augmentation more aggressively to the minority class.
    `minority_class` is the numerical label of the minority class.
    """
    augmented_data = data.clone()
    for i in range(data.size(0)):
        if labels[i] == minority_class:
            # Augment minority class
            # Add Gaussian Noise
            noise = torch.randn_like(data[i]) * noise_level
            augmented_data[i] += noise

            # Time Shift
            shift = np.random.randint(-shift_max, shift_max)
            augmented_data[i] = torch.roll(augmented_data[i], shifts=shift, dims=-1)

    return augmented_data

# --- Assume X_train, y_train, X_valid, y_valid are loaded and prepared ---
# Example placeholder data if you run this standalone
# X_train = np.random.randn(1121, 8, 250)
# y_train = np.random.choice(['Left', 'Right'], 1121, p=[0.7, 0.3]) # Imbalanced example
# X_valid = np.random.randn(24, 8, 250)
# y_valid = np.random.choice(['Left', 'Right'], 24)


# Apply CSP with TIME-SERIES output
csp = CSP(n_components=4, transform_into='csp_space')
X_train_csp = csp.fit_transform(X_train, y_train)
X_valid_csp = csp.transform(X_valid)

# Reshape data for EEGNet
X_train_dl = X_train_csp[:, np.newaxis, :, :]
X_valid_dl = X_valid_csp[:, np.newaxis, :, :]

# Normalize each channel independently
def normalize_channels(data):
    scaler = StandardScaler()
    normalized = np.zeros_like(data)
    for i in range(data.shape[0]):
        for j in range(data.shape[2]): # Iterate through CSP components
            channel_data = data[i, 0, j, :]
            normalized[i, 0, j, :] = scaler.fit_transform(channel_data.reshape(-1, 1)).flatten()
    return normalized

X_train_norm = normalize_channels(X_train_dl)
X_valid_norm = normalize_channels(X_valid_dl)

# Convert to PyTorch tensors
train_tensor = torch.tensor(X_train_norm, dtype=torch.float32)
valid_tensor = torch.tensor(X_valid_norm, dtype=torch.float32)
train_labels_numerical = np.array([1 if label == 'Right' else 0 for label in y_train])
valid_labels_numerical = np.array([1 if label == 'Right' else 0 for label in y_valid])

train_labels = torch.tensor(train_labels_numerical, dtype=torch.long)
valid_labels = torch.tensor(valid_labels_numerical, dtype=torch.long)

# Create DataLoaders
train_dataset = TensorDataset(train_tensor, train_labels)
valid_dataset = TensorDataset(valid_tensor, valid_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32)

# --- 2. CLASS IMBALANCE FIX: Calculate class weights ---
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels_numerical), y=train_labels_numerical)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)
print(f"Calculated class weights: {class_weights_tensor.cpu().numpy()}")

# Initialize model, loss, and optimizer
model = EEGNet(num_channels=4, num_classes=2, sampling_rate=250).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-3) # Slightly increased weight decay
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=8, verbose=True) # Increased patience

# Training loop with Early Stopping
def train_model(model, train_loader, valid_loader, epochs=100):
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'lr': [], 'epoch_time': []}

    # --- 3. OVERFITTING FIX: Early Stopping parameters ---
    best_val_acc = 0.0
    patience = 15  # More aggressive early stopping
    patience_counter = 0
    best_model_wts = None

    for epoch in range(epochs):
        epoch_start = time.time()

        # Training phase
        model.train()
        running_loss, correct_train, total_train = 0.0, 0, 0
        train_iter = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Train]', leave=False)
        for inputs, labels in train_iter:
            # Apply class-wise augmentation
            inputs = augment_data_classwise(inputs, labels, minority_class=1).to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
            train_iter.set_postfix({'loss': running_loss/total_train, 'acc': correct_train/total_train})

        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = correct_train / total_train
        history['train_loss'].append(epoch_train_loss)
        history['train_acc'].append(epoch_train_acc)

        # Validation phase
        model.eval()
        val_loss, correct_val, total_val = 0.0, 0, 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            val_iter = tqdm(valid_loader, desc=f'Epoch {epoch+1}/{epochs} [Valid]', leave=False)
            for inputs, labels in val_iter:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                val_iter.set_postfix({'loss': val_loss/total_val, 'acc': correct_val/total_val})

        epoch_val_loss = val_loss / len(valid_loader.dataset)
        epoch_val_acc = correct_val / total_val
        history['val_loss'].append(epoch_val_loss)
        history['val_acc'].append(epoch_val_acc)

        current_lr = optimizer.param_groups[0]['lr']
        history['lr'].append(current_lr)
        scheduler.step(epoch_val_acc)
        epoch_time = time.time() - epoch_start
        history['epoch_time'].append(epoch_time)

        print(f'\nEpoch {epoch+1}/{epochs} Summary:')
        print(f'  Time: {epoch_time:.2f}s | LR: {current_lr:.6f}')
        print(f'  Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.4f}')
        print(f'  Val Loss:   {epoch_val_loss:.4f} | Val Acc:   {epoch_val_acc:.4f}')

        # Early Stopping Logic based on validation accuracy
        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            patience_counter = 0
            best_model_wts = copy.deepcopy(model.state_dict())
            print(f"  Validation accuracy improved. Saving best model.")
        else:
            patience_counter += 1
            print(f"  Validation accuracy did not improve. Patience: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print("\nEarly stopping triggered.")
            break

    # Load best model weights
    if best_model_wts:
        model.load_state_dict(best_model_wts)

    return all_preds, all_labels, history

# Train the model
print("Training EEGNet model with class-wise augmentation and enhanced regularization...")
preds, true_labels, history = train_model(model, train_loader, valid_loader, epochs=100)

# Convert numerical labels back to string
label_map = {0: 'Left', 1: 'Right'}
str_preds = [label_map[p] for p in preds]
str_true = [label_map[l] for l in true_labels]

# Print final classification report
print("\nFinal Classification Report:")
print(classification_report(str_true, str_preds))

# Print predictions with confidence
model.eval()
confidences = []
with torch.no_grad():
    for inputs, _ in valid_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        probs = torch.softmax(outputs, dim=1)
        conf, _ = torch.max(probs, dim=1)
        confidences.extend(conf.cpu().numpy())

results = pd.DataFrame({
    'True Label': str_true,
    'Predicted': str_preds,
    'Confidence': confidences
})
print("\nValidation set predictions with confidence (CSP + EEGNet):")
print(results)

# Save model
torch.save(model.state_dict(), 'csp_eegnet_model_v2.pth')
print("Model saved to 'csp_eegnet_model_v2.pth'")

# Print training summary
print("\nTraining Summary:")
if history['val_acc']:
    print(f"Best Validation Accuracy: {max(history['val_acc']):.4f}")
print(f"Total Training Time: {sum(history['epoch_time']):.2f} seconds")
if history['epoch_time']:
    print(f"Average Epoch Time: {np.mean(history['epoch_time']):.2f} seconds")
if history['lr']:
    print(f"Final Learning Rate: {history['lr'][-1]:.6f}")

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from collections import Counter

# --- 1. Configuration ---
# Define the base path for the dataset on Kaggle
BASE_PATH = '/kaggle/input/mtcaic3'
# Target time window in seconds
WINDOW = (5.75, 6.75)
# EEG Sampling Frequency
FS = 250
# Subset of channels relevant to motor imagery
# FZ, C3, CZ, C4, PZ, PO7, OZ, PO8 are the channels
# C3, C4, Cz are most relevant for motor cortex activity
CHANNELS = ['C3', 'CZ', 'C4', 'PZ']
# Calculate number of samples in the window
START_SAMP = int(WINDOW[0] * FS)
END_SAMP = int(WINDOW[1] * FS)
N_SAMPLES_WINDOW = END_SAMP - START_SAMP

# --- 2. Data Loading and Preprocessing Functions ---

def load_trial_data(row, base_path, task_folder):
    """Loads a trial's full EEG data from the correct folder (train/validation/test)."""
    # Construct the path to the specific EEGdata.csv file
    eeg_path = os.path.join(base_path, row['task'], task_folder, row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
    eeg_data = pd.read_csv(eeg_path)
    
    # Calculate indices for the specific trial
    trial_num = int(row['trial'])
    samples_per_trial = 2250 if row['task'] == 'MI' else 1750
        
    start_idx = (trial_num - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial
    
    # Extract the trial data
    trial_data = eeg_data.iloc[start_idx:end_idx]
    return trial_data

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data, axis=0)
    return y

# --- 3. Main Data Loading and Filtering Loop ---

print("Loading and preprocessing training data...")
# Load the index file
train_df = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))

X_raw = []
y_raw = []

# Process only Motor Imagery (MI) tasks from the training set
mi_train_df = train_df[train_df['task'] == 'MI'].reset_index(drop=True)

for _, row in mi_train_df.iterrows():
    # Load the full data for the trial
    trial_full_data = load_trial_data(row, BASE_PATH, 'train')
    
    # Extract the specified time window
    trial_window_data = trial_full_data.iloc[START_SAMP:END_SAMP]
    
    # Select only the relevant channels
    X_trial = trial_window_data[CHANNELS].values
    
    # Apply bandpass filter (mu and beta rhythms for MI are typically 8-30 Hz)
    X_trial_filtered = butter_bandpass_filter(X_trial, lowcut=8.0, highcut=30.0, fs=FS)
    
    X_raw.append(X_trial_filtered)
    # Encode labels: 0 for Left, 1 for Right
    y_raw.append(0 if row['label'] == 'Left' else 1)

X_raw = np.array(X_raw)
y_raw = np.array(y_raw)

print(f"Initial raw data shape: {X_raw.shape}")
print(f"Initial raw label distribution: {Counter(y_raw)}")

# --- 4. Contralateral Pattern Filtering ---
print("\nFiltering trials based on contralateral activity...")
X_filtered = []
y_filtered = []

# Channel indices in our CHANNELS list: C3 is 0, C4 is 2
c3_idx, c4_idx = 0, 2 

for i in range(X_raw.shape[0]):
    trial_data = X_raw[i]
    label = y_raw[i]
    
    # Calculate power (mean of squared amplitudes)
    c3_power = np.mean(trial_data[:, c3_idx]**2)
    c4_power = np.mean(trial_data[:, c4_idx]**2)
    
    # Keep trials with expected contralateral activity:
    # Imagined Right Hand (label 1) -> Higher power in Left Hemisphere (C3)
    # Imagined Left Hand (label 0) -> Higher power in Right Hemisphere (C4)
    if (label == 1 and c3_power > c4_power) or \
       (label == 0 and c4_power > c3_power):
        X_filtered.append(trial_data)
        y_filtered.append(label)

X_filtered = np.array(X_filtered)
y_filtered = np.array(y_filtered)

print(f"Shape after filtering: {X_filtered.shape}")
print(f"Label distribution after filtering: {Counter(y_filtered)}")

# --- 5. Class-wise Augmentation for Imbalance ---
print("\nHandling class imbalance with augmentation...")

def augment_minority_class(X, y):
    """Augments the minority class by adding scaled Gaussian noise."""
    class_counts = Counter(y)
    if not class_counts or len(class_counts) < 2:
        print("Not enough classes to augment.")
        return X, y

    minority_class = min(class_counts, key=class_counts.get)
    majority_class = max(class_counts, key=class_counts.get)
    
    n_majority = class_counts[majority_class]
    n_minority = class_counts[minority_class]

    if n_majority == n_minority:
        print("Classes are already balanced.")
        return X, y

    n_to_generate = n_majority - n_minority
    print(f"Balancing classes. Augmenting class '{minority_class}' with {n_to_generate} new samples.")
    
    minority_indices = np.where(y == minority_class)[0]
    
    # Generate new samples from the minority class
    random_indices = np.random.choice(minority_indices, size=n_to_generate, replace=True)
    
    X_minority_original = X[random_indices]
    
    # Add scaled Gaussian noise
    noise_factor = 0.1 
    noise = np.random.normal(0, np.std(X_minority_original) * noise_factor, X_minority_original.shape)
    X_augmented = X_minority_original + noise
    
    # Append augmented data to the original dataset
    X_balanced = np.concatenate((X, X_augmented), axis=0)
    y_augmented = np.full(n_to_generate, minority_class)
    y_balanced = np.concatenate((y, y_augmented), axis=0)
    
    return X_balanced, y_balanced

X_balanced, y_balanced = augment_minority_class(X_filtered, y_filtered)

print(f"Shape after augmentation: {X_balanced.shape}")
print(f"Label distribution after augmentation: {Counter(y_balanced)}")


# --- 6. Final Preparation and Model Training ---
print("\nPreparing data for the model...")

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_balanced, y_balanced, test_size=0.2, random_state=42, stratify=y_balanced
)

# One-hot encode the labels for the neural network
y_train_cat = to_categorical(y_train, num_classes=2)
y_val_cat = to_categorical(y_val, num_classes=2)

# Define the input shape for the model
input_shape = (X_train.shape[1], X_train.shape[2]) # (n_samples_window, n_channels)

# Define the deep learning model
model = Sequential([
    # Input Layer
    Conv1D(filters=32, kernel_size=10, activation='relu', padding='same', input_shape=input_shape),
    BatchNormalization(),
    
    # Spatial Filtering Layer
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),

    # Temporal Feature Extraction Layer
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    
    # Classifier Head
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax') # 2 classes: Left, Right
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Define callbacks for robust training
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=0.00001)

print("\nStarting model training...")
# Train the model
history = model.fit(
    X_train, y_train_cat,
    epochs=100,
    batch_size=32,
    validation_data=(X_val, y_val_cat),
    callbacks=[early_stopping, reduce_lr],
    verbose=2
)

print("\nTraining finished.")
# Evaluate final model on validation set
loss, accuracy = model.evaluate(X_val, y_val_cat, verbose=0)
print(f"\nFinal Validation Accuracy: {accuracy*100:.2f}%")

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from collections import Counter

# --- Import for evaluation ---
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. Configuration ---
# Define the base path for the dataset on Kaggle
BASE_PATH = '/kaggle/input/mtcaic3'
# Target time window in seconds
WINDOW = (5.75, 6.75)
# EEG Sampling Frequency
FS = 250
# Subset of channels relevant to motor imagery
CHANNELS = ['C3', 'CZ', 'C4', 'PZ']
# Calculate number of samples in the window
START_SAMP = int(WINDOW[0] * FS)
END_SAMP = int(WINDOW[1] * FS)
N_SAMPLES_WINDOW = END_SAMP - START_SAMP

# --- 2. Data Loading and Preprocessing Functions ---

def load_trial_data(row, base_path, task_folder):
    """Loads a trial's full EEG data from the correct folder (train/validation/test)."""
    eeg_path = os.path.join(base_path, row['task'], task_folder, row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
    eeg_data = pd.read_csv(eeg_path)
    
    trial_num = int(row['trial'])
    samples_per_trial = 2250 if row['task'] == 'MI' else 1750
        
    start_idx = (trial_num - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial
    
    trial_data = eeg_data.iloc[start_idx:end_idx]
    return trial_data

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data, axis=0)
    return y

# --- 3. Main Data Loading and Filtering Loop ---

print("Loading and preprocessing training data...")
train_df = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
X_raw = []
y_raw = []
mi_train_df = train_df[train_df['task'] == 'MI'].reset_index(drop=True)

for _, row in mi_train_df.iterrows():
    trial_full_data = load_trial_data(row, BASE_PATH, 'train')
    trial_window_data = trial_full_data.iloc[START_SAMP:END_SAMP]
    X_trial = trial_window_data[CHANNELS].values
    X_trial_filtered = butter_bandpass_filter(X_trial, lowcut=8.0, highcut=30.0, fs=FS)
    X_raw.append(X_trial_filtered)
    y_raw.append(0 if row['label'] == 'Left' else 1)

X_raw = np.array(X_raw)
y_raw = np.array(y_raw)

# --- 4. Contralateral Pattern Filtering ---
print("\nFiltering trials based on contralateral activity...")
X_filtered = []
y_filtered = []
c3_idx, c4_idx = 0, 2 

for i in range(X_raw.shape[0]):
    trial_data = X_raw[i]
    label = y_raw[i]
    c3_power = np.mean(trial_data[:, c3_idx]**2)
    c4_power = np.mean(trial_data[:, c4_idx]**2)
    
    if (label == 1 and c3_power > c4_power) or \
       (label == 0 and c4_power > c3_power):
        X_filtered.append(trial_data)
        y_filtered.append(label)

X_filtered = np.array(X_filtered)
y_filtered = np.array(y_filtered)

print(f"Shape of data before augmentation: {X_filtered.shape}")
print(f"Label distribution before augmentation: {Counter(y_filtered)}")

# --- 5. Class-wise Augmentation for Imbalance ---
print("\nHandling class imbalance with augmentation...")

def augment_minority_class(X, y):
    """Augments the minority class by adding scaled Gaussian noise."""
    class_counts = Counter(y)
    if not class_counts or len(class_counts) < 2:
        return X, y

    minority_class = min(class_counts, key=class_counts.get)
    majority_class = max(class_counts, key=class_counts.get)
    
    n_majority = class_counts[majority_class]
    n_minority = class_counts[minority_class]

    if n_majority == n_minority:
        print("Classes are already balanced.")
        return X, y

    n_to_generate = n_majority - n_minority
    print(f"Balancing classes. Augmenting class '{minority_class}' with {n_to_generate} new samples.")
    
    minority_indices = np.where(y == minority_class)[0]
    random_indices = np.random.choice(minority_indices, size=n_to_generate, replace=True)
    X_minority_original = X[random_indices]
    
    noise_factor = 0.1 
    noise = np.random.normal(0, np.std(X_minority_original) * noise_factor, X_minority_original.shape)
    X_augmented = X_minority_original + noise
    
    X_balanced = np.concatenate((X, X_augmented), axis=0)
    y_augmented = np.full(n_to_generate, minority_class)
    y_balanced = np.concatenate((y, y_augmented), axis=0)
    
    return X_balanced, y_balanced

X_balanced, y_balanced = augment_minority_class(X_filtered, y_filtered)

print(f"Shape of data after augmentation: {X_balanced.shape}")
print(f"Label distribution after augmentation: {Counter(y_balanced)}")

# --- 6. Final Preparation and Model Training ---
print("\nPreparing data for the model...")

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_balanced, y_balanced, test_size=0.2, random_state=42, stratify=y_balanced
)

# --- EDITED SECTION: Print shapes of data fed into the model ---
print("\n--- Data Fed Into Model ---")
print(f"Training data shape (X_train): {X_train.shape}")
print(f"Training labels shape (y_train): {y_train.shape}")
print(f"Validation data shape (X_val): {X_val.shape}")
print(f"Validation labels shape (y_val): {y_val.shape}")
print("---------------------------\n")

# One-hot encode the labels for the neural network
y_train_cat = to_categorical(y_train, num_classes=2)
y_val_cat = to_categorical(y_val, num_classes=2)

# Define the input shape for the model
input_shape = (X_train.shape[1], X_train.shape[2])

model = Sequential([
    Conv1D(filters=32, kernel_size=10, activation='relu', padding='same', input_shape=input_shape),
    BatchNormalization(),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=0.00001)

print("\nStarting model training...")
# Train the model with verbose=1 to show training logs
history = model.fit(
    X_train, y_train_cat,
    epochs=100,
    batch_size=32,
    validation_data=(X_val, y_val_cat),
    callbacks=[early_stopping, reduce_lr],
    verbose=1 # EDITED: Changed from 2 to 1 for progress bar logs
)

print("\nTraining finished.")
loss, accuracy = model.evaluate(X_val, y_val_cat, verbose=0)
print(f"\nFinal Validation Accuracy: {accuracy*100:.2f}%")

# --- 7. EDITED SECTION: Model Evaluation ---
print("\n--- Model Evaluation on Validation Set ---")

# Get model predictions
y_pred_probs = model.predict(X_val)
y_pred = np.argmax(y_pred_probs, axis=1)

# Define class names (0=Left, 1=Right)
class_names = ['Left', 'Right']

# 1. Classification Report
print("\nClassification Report:")
print(classification_report(y_val, y_pred, target_names=class_names))

# 2. Confusion Matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_val, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix on Validation Data')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.signal import butter, lfilter
from collections import Counter

from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


# --- 1. Configuration ---
# Define the base path for the dataset on Kaggle
BASE_PATH = '/kaggle/input/mtcaic3'
# Target time window in seconds
WINDOW = (5.75, 6.75)
# EEG Sampling Frequency
FS = 250
# Subset of channels relevant to motor imagery
CHANNELS = ['C3', 'CZ', 'C4', 'PZ']
# Calculate number of samples in the window
START_SAMP = int(WINDOW[0] * FS)
END_SAMP = int(WINDOW[1] * FS)


# --- 2. Data Loading and Preprocessing Functions ---

def load_trial_data(row, base_path, task_folder):
    """Loads a trial's full EEG data from the correct folder (train/validation/test)."""
    eeg_path = os.path.join(base_path, row['task'], task_folder, row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
    eeg_data = pd.read_csv(eeg_path)
    
    trial_num = int(row['trial'])
    samples_per_trial = 2250
        
    start_idx = (trial_num - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial
    
    return eeg_data.iloc[start_idx:end_idx]

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return lfilter(b, a, data, axis=0)

def process_data_from_df(df, base_path, task_folder):
    """
    Helper function to load, filter, and preprocess data from a given dataframe (train or validation).
    Processes only MI tasks.
    """
    X_data, y_data = [], []
    mi_df = df[df['task'] == 'MI'].reset_index(drop=True)
    
    for _, row in mi_df.iterrows():
        trial_full_data = load_trial_data(row, base_path, task_folder)
        trial_window_data = trial_full_data.iloc[START_SAMP:END_SAMP]
        X_trial = trial_window_data[CHANNELS].values
        X_trial_filtered = butter_bandpass_filter(X_trial, lowcut=8.0, highcut=30.0, fs=FS)
        X_data.append(X_trial_filtered)
        y_data.append(0 if row['label'] == 'Left' else 1)
        
    return np.array(X_data), np.array(y_data)


# --- 3. Load Official Train and Validation Data ---
print("--- Loading and Processing Data ---")
train_df = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
validation_df = pd.read_csv(os.path.join(BASE_PATH, 'validation.csv'))

X_train, y_train = process_data_from_df(train_df, BASE_PATH, 'train')
X_val, y_val = process_data_from_df(validation_df, BASE_PATH, 'validation')

print(f"\nLoaded {X_train.shape[0]} raw training samples.")
print(f"Loaded {X_val.shape[0]} raw validation samples from new subjects.")


# --- 4. Contralateral Pattern Filtering ---
def filter_by_contralateral_pattern(X, y):
    """Keeps only trials that follow the expected contralateral MI pattern."""
    X_filtered, y_filtered = [], []
    c3_idx, c4_idx = 0, 2
    for i in range(X.shape[0]):
        power_c3 = np.mean(X[i, :, c3_idx]**2)
        power_c4 = np.mean(X[i, :, c4_idx]**2)
        
        if (y[i] == 1 and power_c3 > power_c4) or \
           (y[i] == 0 and power_c4 > power_c3):
            X_filtered.append(X[i])
            y_filtered.append(y[i])
            
    return np.array(X_filtered), np.array(y_filtered)

print("\nFiltering training trials based on contralateral patterns...")
X_train_filtered, y_train_filtered = filter_by_contralateral_pattern(X_train, y_train)
print(f"Kept {len(X_train_filtered)} of {len(X_train)} training samples after filtering.")

print("\nFiltering validation trials based on contralateral patterns...")
X_val_filtered, y_val_filtered = filter_by_contralateral_pattern(X_val, y_val)
print(f"Kept {len(X_val_filtered)} of {len(X_val)} validation samples after filtering.")


# --- 5. Augment Filtered Training Data Only ---
def augment_training_data(X, y):
    """Augments the minority class in the training set by adding scaled Gaussian noise."""
    class_counts = Counter(y)
    if not class_counts or len(class_counts) < 2:
        return X, y
    
    minority_class = min(class_counts, key=class_counts.get)
    n_to_generate = max(class_counts.values()) - min(class_counts.values())

    if n_to_generate == 0:
        print("\nTraining data is already balanced. No augmentation needed.")
        return X, y

    print(f"\nBalancing training data. Augmenting class '{minority_class}' with {n_to_generate} new samples.")
    
    minority_indices = np.where(y == minority_class)[0]
    random_indices = np.random.choice(minority_indices, size=n_to_generate, replace=True)
    X_minority_original = X[random_indices]
    noise_factor = 0.1 
    noise = np.random.normal(0, np.std(X_minority_original) * noise_factor, X_minority_original.shape)
    X_augmented = X_minority_original + noise
    
    return np.concatenate((X, X_augmented)), np.concatenate((y, np.full(n_to_generate, minority_class)))

X_train_aug, y_train_aug = augment_training_data(X_train_filtered, y_train_filtered)


# --- 6. Prepare Data for Model and Print Final Shapes ---
print("\n--- Final Data Shapes Fed Into Model ---")
print(f"Augmented Training Data: {X_train_aug.shape}")
print(f"Augmented Training Labels: {y_train_aug.shape} (Distribution: {Counter(y_train_aug)})")
print(f"Filtered Validation Data: {X_val_filtered.shape}")
print(f"Filtered Validation Labels: {y_val_filtered.shape} (Distribution: {Counter(y_val_filtered)})")
print("----------------------------------------\n")

# One-hot encode the labels for the neural network
y_train_cat = to_categorical(y_train_aug, num_classes=2)
y_val_cat = to_categorical(y_val_filtered, num_classes=2)


# --- 7. Model Definition and Training ---
input_shape = (X_train_aug.shape[1], X_train_aug.shape[2])

model = Sequential([
    Conv1D(filters=32, kernel_size=10, activation='relu', padding='same', input_shape=input_shape),
    BatchNormalization(),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=1e-5)
]

print("\nStarting model training...")
# Check if there is data to validate on
if X_val_filtered.shape[0] > 0:
    history = model.fit(
        X_train_aug, y_train_cat,
        epochs=100,
        batch_size=32,
        validation_data=(X_val_filtered, y_val_cat),
        callbacks=callbacks,
        verbose=1
    )
else:
    print("Skipping training with validation as the filtered validation set is empty.")
    history = model.fit(
        X_train_aug, y_train_cat,
        epochs=100,
        batch_size=32,
        callbacks=[],
        verbose=1
    )


# --- 8. Final Evaluation on Filtered Validation Set ---
print("\n--- Model Evaluation on Filtered Validation Set (Unseen Subjects) ---")

if X_val_filtered.shape[0] > 0:
    loss, accuracy = model.evaluate(X_val_filtered, y_val_cat, verbose=0)
    print(f"\nFinal Validation Accuracy on filtered data from unseen subjects: {accuracy*100:.2f}%")
    
    y_pred_probs = model.predict(X_val_filtered)
    y_pred = np.argmax(y_pred_probs, axis=1)
    class_names = ['Left', 'Right']

    # === FIX IMPLEMENTED HERE ===
    
    # Display Classification Report
    print("\nClassification Report:")
    # Using the 'labels' parameter prevents the error if a class is missing.
    # 'zero_division=0' handles cases where a metric (like precision) can't be calculated.
    print(classification_report(y_val_filtered, y_pred, target_names=class_names, labels=[0, 1], zero_division=0))

    # Display Confusion Matrix
    print("\nConfusion Matrix:")
    # Using the 'labels' parameter here ensures the matrix is always 2x2.
    cm = confusion_matrix(y_val_filtered, y_pred, labels=[0, 1])
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix on Filtered Validation Set')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()
    
else:
    print("\nNo data in the filtered validation set to evaluate.")

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.signal import butter, lfilter
from mne.decoding import CSP
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

# --- 1. CONFIGURATION AND SETUP ---
# Kaggle environment path
BASE_PATH = '/kaggle/input/mtcaic3'

# EEG constants
SAMPLE_RATE = 250  # Hz
MI_CHANNELS = ['C3', 'Cz', 'C4'] # Core channels for Motor Imagery
ALL_EEG_CHANNELS = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
WINDOW = (5.75, 6.75)  # Target time window in seconds
MU_BAND = (8, 12) # Frequency band for contralateral check (Hz)

# Model parameters
CSP_COMPONENTS = 4 # Number of CSP components to use as features
EPOCHS = 100
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2

# --- 2. HELPER FUNCTIONS ---

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data, axis=0)
    return y

def load_trial_data(row, base_path, split):
    """Loads raw EEG data for a single trial specified by a metadata row."""
    eeg_path = os.path.join(base_path, row['task'], split, row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
    eeg_data = pd.read_csv(eeg_path)
    
    trial_num = int(row['trial'])
    samples_per_trial = 2250 # MI: 9s * 250Hz
    
    start_idx = (trial_num - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial
    
    trial_data = eeg_data.iloc[start_idx:end_idx]
    return trial_data

def check_contralateral_pattern(trial_data_window, label):
    """
    Checks if a trial exhibits the expected contralateral ERD pattern.
    - Left hand MI should show lower power in the right hemisphere (C4) vs left (C3).
    - Right hand MI should show lower power in the left hemisphere (C3) vs right (C4).
    """
    # Filter data for mu-band frequencies
    c3_filtered = bandpass_filter(trial_data_window['C3'], MU_BAND[0], MU_BAND[1], SAMPLE_RATE)
    c4_filtered = bandpass_filter(trial_data_window['C4'], MU_BAND[0], MU_BAND[1], SAMPLE_RATE)

    # Calculate power (variance)
    power_c3 = np.var(c3_filtered)
    power_c4 = np.var(c4_filtered)

    if label == 'Left':
        return power_c4 < power_c3
    elif label == 'Right':
        return power_c3 < power_c4
    else:
        return False # Should not happen for MI tasks

def augment_trial(trial_data, label):
    """Augments data by flipping channels and adding noise."""
    # 1. Channel Flipping
    flipped_data = trial_data.copy()
    c3_data = flipped_data['C3'].copy()
    flipped_data['C3'] = flipped_data['C4']
    flipped_data['C4'] = c3_data
    flipped_label = 'Right' if label == 'Left' else 'Left'
    
    # 2. Noise Addition
    noise = np.random.normal(0, 0.1 * np.std(trial_data), trial_data.shape)
    noisy_data = trial_data + noise
    
    return [(flipped_data, flipped_label), (noisy_data, label)]


def create_dl_model(input_shape):
    """Creates a simple Deep Learning model for classification."""
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid') # Binary classification
    ])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# --- 3. DATA LOADING AND PREPROCESSING ---

print("Loading metadata...")
train_df = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
validation_df = pd.read_csv(os.path.join(BASE_PATH, 'validation.csv'))

# Filter for Motor Imagery (MI) tasks only
mi_train_df = train_df[train_df['task'] == 'MI'].reset_index(drop=True)
mi_validation_df = validation_df[validation_df['task'] == 'MI'].reset_index(drop=True)

def process_dataframe(df, split):
    """
    Loads, filters, and processes EEG data based on the contralateral pattern.
    """
    print(f"\nProcessing {split} data...")
    X, y = [], []
    
    # Calculate sample indices for the time window
    start_sample = int(WINDOW[0] * SAMPLE_RATE)
    end_sample = int(WINDOW[1] * SAMPLE_RATE)

    dropped_trials = 0
    for i, row in df.iterrows():
        # Load data for the entire trial
        full_trial_data = load_trial_data(row, BASE_PATH, split)
        
        # Extract the specific time window
        windowed_data = full_trial_data.iloc[start_sample:end_sample]
        
        # Check for contralateral pattern
        if check_contralateral_pattern(windowed_data, row['label']):
            # Use only the specified MI channels
            trial_channels = windowed_data[MI_CHANNELS]
            X.append(trial_channels.to_numpy())
            y.append(row['label'])
        else:
            dropped_trials += 1
            
    print(f"Kept {len(X)} trials. Dropped {dropped_trials} trials that did not fit the contralateral pattern.")
    return np.array(X), np.array(y)

# Process training and validation sets
X_train_raw, y_train_raw = process_dataframe(mi_train_df, 'train')
X_val_filtered, y_val_filtered = process_dataframe(mi_validation_df, 'validation')

# --- 4. DATA AUGMENTATION (ON TRAINING SET ONLY) ---
print("\nAugmenting training data...")
X_train_aug, y_train_aug = [], []
for trial, label in zip(X_train_raw, y_train_raw):
    # Add original trial
    X_train_aug.append(trial)
    y_train_aug.append(label)
    
    # Create a DataFrame to use augment function
    trial_df = pd.DataFrame(trial, columns=MI_CHANNELS)
    
    # Add augmented versions
    augmented_versions = augment_trial(trial_df, label)
    for aug_data, aug_label in augmented_versions:
        X_train_aug.append(aug_data.to_numpy())
        y_train_aug.append(aug_label)

X_train_aug = np.array(X_train_aug)
y_train_aug = np.array(y_train_aug)
print(f"Original training size: {len(X_train_raw)}. Augmented training size: {len(X_train_aug)}")

# --- 5. LABEL ENCODING and CLASS WEIGHTS ---
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train_aug)
y_val_encoded = le.transform(y_val_filtered)

# Handle class imbalance after filtering and augmentation
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)
class_weights_dict = dict(enumerate(class_weights))
print(f"\nClass weights for handling imbalance: {class_weights_dict}")

# --- 6. CSP FEATURE EXTRACTION ---
print("\nFitting CSP and transforming data...")
# Reshape data for MNE: (n_epochs, n_channels, n_times)
X_train_csp_input = X_train_aug.transpose(0, 2, 1)
X_val_csp_input = X_val_filtered.transpose(0, 2, 1)

# Initialize and fit CSP
csp = CSP(n_components=CSP_COMPONENTS, reg=None, log=True, norm_trace=False)
csp.fit(X_train_csp_input, y_train_encoded)

# Transform data into CSP features
X_train_features = csp.transform(X_train_csp_input)
X_val_features = csp.transform(X_val_csp_input)
print(f"Shape of data after CSP transformation: {X_train_features.shape}")

# --- 7. DEEP LEARNING MODEL TRAINING ---
print("\nBuilding and training the Deep Learning model...")
model = create_dl_model(input_shape=X_train_features.shape[1])
model.summary()

early_stopping = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)

history = model.fit(
    X_train_features,
    y_train_encoded,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val_features, y_val_encoded),
    class_weight=class_weights_dict,
    callbacks=[early_stopping],
    verbose=2
)

# --- 8. EVALUATION ---
print("\nEvaluating model on the filtered validation set...")
loss, accuracy = model.evaluate(X_val_features, y_val_encoded)
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Loss: {loss:.4f}")

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.signal import butter, lfilter
from mne.decoding import CSP
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# --- 1. CONFIGURATION AND SETUP ---
# Kaggle environment path
BASE_PATH = '/kaggle/input/mtcaic3'

# EEG constants
SAMPLE_RATE = 250  # Hz
MI_CHANNELS = ['C3', 'Cz', 'C4'] # Core channels for Motor Imagery
ALL_EEG_CHANNELS = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
WINDOW = (5.75, 6.75)  # Target time window in seconds
MU_BAND = (8, 12) # Frequency band for contralateral check (Hz)

# Model parameters
CSP_COMPONENTS = 4 # Number of CSP components to use as features
EPOCHS = 100
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2

# --- 2. HELPER FUNCTIONS ---

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data, axis=0)
    return y

def load_trial_data(row, base_path, split):
    """Loads raw EEG data for a single trial specified by a metadata row."""
    eeg_path = os.path.join(base_path, row['task'], split, row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
    eeg_data = pd.read_csv(eeg_path)
    
    trial_num = int(row['trial'])
    samples_per_trial = 2250 # MI: 9s * 250Hz
    
    start_idx = (trial_num - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial
    
    trial_data = eeg_data.iloc[start_idx:end_idx]
    return trial_data

def check_contralateral_pattern(trial_data_window, label):
    """
    Checks if a trial exhibits the expected contralateral ERD pattern.
    - Left hand MI should show lower power in the right hemisphere (C4) vs left (C3).
    - Right hand MI should show lower power in the left hemisphere (C3) vs right (C4).
    """
    # Filter data for mu-band frequencies
    c3_filtered = bandpass_filter(trial_data_window['C3'], MU_BAND[0], MU_BAND[1], SAMPLE_RATE)
    c4_filtered = bandpass_filter(trial_data_window['C4'], MU_BAND[0], MU_BAND[1], SAMPLE_RATE)

    # Calculate power (variance)
    power_c3 = np.var(c3_filtered)
    power_c4 = np.var(c4_filtered)

    if label == 'Left':
        return power_c4 < power_c3
    elif label == 'Right':
        return power_c3 < power_c4
    else:
        return False # Should not happen for MI tasks

def augment_trial(trial_data, label):
    """Augments data by flipping channels and adding noise."""
    # 1. Channel Flipping
    flipped_data = trial_data.copy()
    c3_data = flipped_data['C3'].copy()
    flipped_data['C3'] = flipped_data['C4']
    flipped_data['C4'] = c3_data
    flipped_label = 'Right' if label == 'Left' else 'Left'
    
    # 2. Noise Addition
    noise = np.random.normal(0, 0.1 * np.std(trial_data), trial_data.shape)
    noisy_data = trial_data + noise
    
    return [(flipped_data, flipped_label), (noisy_data, label)]


def create_dl_model(input_shape):
    """Creates a simple Deep Learning model for classification."""
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid') # Binary classification
    ])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# --- 3. DATA LOADING AND PREPROCESSING ---

print("Loading metadata...")
train_df = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
validation_df = pd.read_csv(os.path.join(BASE_PATH, 'validation.csv'))

# Filter for Motor Imagery (MI) tasks only
mi_train_df = train_df[train_df['task'] == 'MI'].reset_index(drop=True)
mi_validation_df = validation_df[validation_df['task'] == 'MI'].reset_index(drop=True)

def process_dataframe(df, split):
    """
    Loads, filters, and processes EEG data based on the contralateral pattern.
    """
    print(f"\nProcessing {split} data...")
    X, y = [], []
    
    # Calculate sample indices for the time window
    start_sample = int(WINDOW[0] * SAMPLE_RATE)
    end_sample = int(WINDOW[1] * SAMPLE_RATE)

    dropped_trials = 0
    for i, row in df.iterrows():
        # Load data for the entire trial
        full_trial_data = load_trial_data(row, BASE_PATH, split)
        
        # Extract the specific time window
        windowed_data = full_trial_data.iloc[start_sample:end_sample]
        
        # Check for contralateral pattern
        if check_contralateral_pattern(windowed_data, row['label']):
            # Use only the specified MI channels
            trial_channels = windowed_data[MI_CHANNELS]
            X.append(trial_channels.to_numpy())
            y.append(row['label'])
        else:
            dropped_trials += 1
            
    print(f"Kept {len(X)} trials. Dropped {dropped_trials} trials that did not fit the contralateral pattern.")
    return np.array(X), np.array(y)

# Process training and validation sets
X_train_raw, y_train_raw = process_dataframe(mi_train_df, 'train')
X_val_filtered, y_val_filtered = process_dataframe(mi_validation_df, 'validation')

# --- 4. DATA AUGMENTATION (ON TRAINING SET ONLY) ---
print("\nAugmenting training data...")
X_train_aug, y_train_aug = [], []
for trial, label in zip(X_train_raw, y_train_raw):
    # Add original trial
    X_train_aug.append(trial)
    y_train_aug.append(label)
    
    # Create a DataFrame to use augment function
    trial_df = pd.DataFrame(trial, columns=MI_CHANNELS)
    
    # Add augmented versions
    augmented_versions = augment_trial(trial_df, label)
    for aug_data, aug_label in augmented_versions:
        X_train_aug.append(aug_data.to_numpy())
        y_train_aug.append(aug_label)

X_train_aug = np.array(X_train_aug)
y_train_aug = np.array(y_train_aug)
print(f"Original training size: {len(X_train_raw)}. Augmented training size: {len(X_train_aug)}")

# --- 5. LABEL ENCODING and CLASS WEIGHTS ---
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train_aug)
y_val_encoded = le.transform(y_val_filtered)

# Handle class imbalance after filtering and augmentation
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)
class_weights_dict = dict(enumerate(class_weights))
print(f"\nClass weights for handling imbalance: {class_weights_dict}")

# --- 6. CSP FEATURE EXTRACTION ---
print("\nFitting CSP and transforming data...")
# Reshape data for MNE: (n_epochs, n_channels, n_times)
X_train_csp_input = X_train_aug.transpose(0, 2, 1)
X_val_csp_input = X_val_filtered.transpose(0, 2, 1)

# Initialize and fit CSP
csp = CSP(n_components=CSP_COMPONENTS, reg=None, log=True, norm_trace=False)
csp.fit(X_train_csp_input, y_train_encoded)

# Transform data into CSP features
X_train_features = csp.transform(X_train_csp_input)
X_val_features = csp.transform(X_val_csp_input)
print(f"Shape of data after CSP transformation: {X_train_features.shape}")

# --- 7. DEEP LEARNING MODEL TRAINING ---
print("\nBuilding and training the Deep Learning model...")
model = create_dl_model(input_shape=X_train_features.shape[1])
model.summary()

early_stopping = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)

history = model.fit(
    X_train_features,
    y_train_encoded,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val_features, y_val_encoded),
    class_weight=class_weights_dict,
    callbacks=[early_stopping],
    verbose=2
)

# --- 8. EVALUATION ---
print("\n--- Basic Model Evaluation ---")
loss, accuracy = model.evaluate(X_val_features, y_val_encoded, verbose=0)
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Loss: {loss:.4f}")

# --- 9. DETAILED EVALUATION AND REPORTING ---
print("\n--- Detailed Performance Analysis ---")
# Get model predictions for the validation set
y_pred_probs = model.predict(X_val_features)
y_pred = (y_pred_probs > 0.5).astype(int).flatten() # Convert probabilities to binary classes (0 or 1)

# Generate and print the classification report
print("\nClassification Report:")
print(classification_report(y_val_encoded, y_pred, target_names=le.classes_))

# Generate and plot the confusion matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_val_encoded, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix for Validation Data')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.signal import butter, lfilter
from mne.decoding import CSP
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# --- 1. CONFIGURATION AND SETUP ---
# Kaggle environment path
BASE_PATH = '/kaggle/input/mtcaic3'

# EEG constants
SAMPLE_RATE = 250  # Hz
MI_CHANNELS = ['C3', 'CZ', 'C4'] # Core channels for Motor Imagery
ALL_EEG_CHANNELS = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
WINDOW = (2.75, 3.75)  # Target time window in seconds
MU_BAND = (8, 12) # Frequency band for contralateral check (Hz)

# Model parameters
CSP_COMPONENTS = 4 # Number of CSP components to use as features
EPOCHS = 100
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2

# --- 2. HELPER FUNCTIONS ---

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data, axis=0)
    return y

def load_trial_data(row, base_path, split):
    """Loads raw EEG data for a single trial specified by a metadata row."""
    eeg_path = os.path.join(base_path, row['task'], split, row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
    eeg_data = pd.read_csv(eeg_path)
    
    trial_num = int(row['trial'])
    samples_per_trial = 2250 # MI: 9s * 250Hz
    
    start_idx = (trial_num - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial
    
    trial_data = eeg_data.iloc[start_idx:end_idx]
    return trial_data

def check_contralateral_pattern(trial_data_window, label):
    """
    Checks if a trial exhibits the expected contralateral ERD pattern.
    - Left hand MI should show lower power in the right hemisphere (C4) vs left (C3).
    - Right hand MI should show lower power in the left hemisphere (C3) vs right (C4).
    """
    # Filter data for mu-band frequencies
    c3_filtered = bandpass_filter(trial_data_window['C3'], MU_BAND[0], MU_BAND[1], SAMPLE_RATE)
    c4_filtered = bandpass_filter(trial_data_window['C4'], MU_BAND[0], MU_BAND[1], SAMPLE_RATE)

    # Calculate power (variance)
    power_c3 = np.var(c3_filtered)
    power_c4 = np.var(c4_filtered)

    if label == 'Left':
        return power_c4 < power_c3
    elif label == 'Right':
        return power_c3 < power_c4
    else:
        return False # Should not happen for MI tasks

def augment_trial(trial_data, label):
    """Augments data by flipping channels and adding noise."""
    # 1. Channel Flipping
    flipped_data = trial_data.copy()
    c3_data = flipped_data['C3'].copy()
    flipped_data['C3'] = flipped_data['C4']
    flipped_data['C4'] = c3_data
    flipped_label = 'Right' if label == 'Left' else 'Left'
    
    # 2. Noise Addition
    noise = np.random.normal(0, 0.1 * np.std(trial_data), trial_data.shape)
    noisy_data = trial_data + noise
    
    return [(flipped_data, flipped_label), (noisy_data, label)]


def create_dl_model(input_shape):
    """Creates a simple Deep Learning model for classification."""
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid') # Binary classification
    ])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# --- 3. DATA LOADING AND PREPROCESSING ---

print("Loading metadata...")
train_df = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
validation_df = pd.read_csv(os.path.join(BASE_PATH, 'validation.csv'))

# Filter for Motor Imagery (MI) tasks only
mi_train_df = train_df[train_df['task'] == 'MI'].reset_index(drop=True)
mi_validation_df = validation_df[validation_df['task'] == 'MI'].reset_index(drop=True)

def process_dataframe(df, split):
    """
    Loads, filters, and processes EEG data based on the contralateral pattern.
    """
    print(f"\nProcessing {split} data...")
    X, y = [], []
    
    # Calculate sample indices for the time window
    start_sample = int(WINDOW[0] * SAMPLE_RATE)
    end_sample = int(WINDOW[1] * SAMPLE_RATE)

    dropped_trials = 0
    for i, row in df.iterrows():
        # Load data for the entire trial
        full_trial_data = load_trial_data(row, BASE_PATH, split)
        
        # Extract the specific time window
        windowed_data = full_trial_data.iloc[start_sample:end_sample]
        
        # Check for contralateral pattern
        if check_contralateral_pattern(windowed_data, row['label']):
            # Use only the specified MI channels
            trial_channels = windowed_data[MI_CHANNELS]
            X.append(trial_channels.to_numpy())
            y.append(row['label'])
        else:
            dropped_trials += 1
            
    print(f"Kept {len(X)} trials. Dropped {dropped_trials} trials that did not fit the contralateral pattern.")
    return np.array(X), np.array(y)

# Process training and validation sets
X_train_raw, y_train_raw = process_dataframe(mi_train_df, 'train')
X_val_filtered, y_val_filtered = process_dataframe(mi_validation_df, 'validation')

# --- 4. DATA AUGMENTATION (ON TRAINING SET ONLY) ---
print("\nAugmenting training data...")
X_train_aug, y_train_aug = [], []
for trial, label in zip(X_train_raw, y_train_raw):
    # Add original trial
    X_train_aug.append(trial)
    y_train_aug.append(label)
    
    # Create a DataFrame to use augment function
    trial_df = pd.DataFrame(trial, columns=MI_CHANNELS)
    
    # Add augmented versions
    augmented_versions = augment_trial(trial_df, label)
    for aug_data, aug_label in augmented_versions:
        X_train_aug.append(aug_data.to_numpy())
        y_train_aug.append(aug_label)

X_train_aug = np.array(X_train_aug)
y_train_aug = np.array(y_train_aug)
print(f"Original training size: {len(X_train_raw)}. Augmented training size: {len(X_train_aug)}")

# --- 5. LABEL ENCODING and CLASS WEIGHTS ---
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train_aug)
y_val_encoded = le.transform(y_val_filtered)

# Handle class imbalance after filtering and augmentation
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)
class_weights_dict = dict(enumerate(class_weights))
print(f"\nClass weights for handling imbalance: {class_weights_dict}")

# --- 6. CSP FEATURE EXTRACTION ---
print("\nFitting CSP and transforming data...")
# Reshape data for MNE: (n_epochs, n_channels, n_times)
X_train_csp_input = X_train_aug.transpose(0, 2, 1)
X_val_csp_input = X_val_filtered.transpose(0, 2, 1)

# Initialize and fit CSP
csp = CSP(n_components=CSP_COMPONENTS, reg=None, log=True, norm_trace=False)
csp.fit(X_train_csp_input, y_train_encoded)

# Transform data into CSP features
X_train_features = csp.transform(X_train_csp_input)
X_val_features = csp.transform(X_val_csp_input)
print(f"Shape of data after CSP transformation: {X_train_features.shape}")

# --- 7. DEEP LEARNING MODEL TRAINING ---
print("\nBuilding and training the Deep Learning model...")
model = create_dl_model(input_shape=X_train_features.shape[1])
model.summary()

early_stopping = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)

history = model.fit(
    X_train_features,
    y_train_encoded,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val_features, y_val_encoded),
    class_weight=class_weights_dict,
    callbacks=[early_stopping],
    verbose=2
)

# --- 8. EVALUATION ---
print("\n--- Basic Model Evaluation ---")
loss, accuracy = model.evaluate(X_val_features, y_val_encoded, verbose=0)
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Loss: {loss:.4f}")

# --- 9. DETAILED EVALUATION AND REPORTING ---
print("\n--- Detailed Performance Analysis ---")
# Get model predictions for the validation set
y_pred_probs = model.predict(X_val_features)
y_pred = (y_pred_probs > 0.5).astype(int).flatten() # Convert probabilities to binary classes (0 or 1)

# Generate and print the classification report
print("\nClassification Report:")
print(classification_report(y_val_encoded, y_pred, target_names=le.classes_))

# Generate and plot the confusion matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_val_encoded, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix for Validation Data')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.signal import butter, lfilter

# --- 1. CONFIGURATION ---
# Kaggle environment path
BASE_PATH = '/kaggle/input/mtcaic3'

# EEG constants
SAMPLE_RATE = 250  # Hz
MU_BAND = (8, 12)  # Frequency band (in Hz) for checking contralateral activity

# Define all the time windows to be tested (in seconds)
TIME_WINDOWS = [
    (1.25, 2.25),
    (2.75, 3.75),
    (3.50, 4.50),
    (4.25, 5.25),
    (5.00, 6.00),
    (5.75, 6.75),
    (6.00, 7.00),
    (7.00, 8.00)
]

# --- 2. HELPER FUNCTIONS ---

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data, axis=0)
    return y

def load_trial_data(row, base_path, split='validation'):
    """Loads raw EEG data for a single trial specified by a metadata row."""
    eeg_path = os.path.join(base_path, row['task'], split, row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
    eeg_data = pd.read_csv(eeg_path)
    
    trial_num = int(row['trial'])
    samples_per_trial = 2250  # MI: 9s * 250Hz
    
    start_idx = (trial_num - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial
    
    trial_data = eeg_data.iloc[start_idx:end_idx]
    return trial_data

def check_contralateral_pattern(trial_data_window, label):
    """
    Checks if a trial exhibits the expected contralateral ERD pattern.
    - Left hand MI should show lower power in the right hemisphere (C4) vs left (C3).
    - Right hand MI should show lower power in the left hemisphere (C3) vs right (C4).
    """
    # Filter data for mu-band frequencies
    c3_filtered = bandpass_filter(trial_data_window['C3'], MU_BAND[0], MU_BAND[1], SAMPLE_RATE)
    c4_filtered = bandpass_filter(trial_data_window['C4'], MU_BAND[0], MU_BAND[1], SAMPLE_RATE)

    # Calculate power (variance)
    power_c3 = np.var(c3_filtered)
    power_c4 = np.var(c4_filtered)

    if label == 'Left':
        return power_c4 < power_c3
    elif label == 'Right':
        return power_c3 < power_c4
    return False

# --- 3. MAIN ANALYSIS SCRIPT ---

print("Loading and filtering validation metadata for MI tasks...")
validation_df = pd.read_csv(os.path.join(BASE_PATH, 'validation.csv'))
mi_validation_df = validation_df[validation_df['task'] == 'MI'].reset_index(drop=True)

trials_with_pattern = []
trial_labels_with_pattern = []

print(f"\nAnalyzing {len(mi_validation_df)} MI trials across {len(TIME_WINDOWS)} time windows...")

# Iterate through each MI trial in the validation set
for index, row in mi_validation_df.iterrows():
    full_trial_data = load_trial_data(row, BASE_PATH)
    
    found_pattern = False
    # Check each defined time window for the pattern
    for start_t, end_t in TIME_WINDOWS:
        start_sample = int(start_t * SAMPLE_RATE)
        end_sample = int(end_t * SAMPLE_RATE)
        
        windowed_data = full_trial_data.iloc[start_sample:end_sample]
        
        # If the pattern is found in any window, mark it and stop checking other windows
        if check_contralateral_pattern(windowed_data, row['label']):
            trials_with_pattern.append(row['id'])
            trial_labels_with_pattern.append(row['label'])
            found_pattern = True
            break  # Move to the next trial once a pattern is found

# --- 4. REPORTING RESULTS ---

total_mi_trials = len(mi_validation_df)
num_trials_with_pattern = len(trials_with_pattern)
percentage_with_pattern = (num_trials_with_pattern / total_mi_trials) * 100 if total_mi_trials > 0 else 0

# Calculate the class distribution of the trials that showed the pattern
class_distribution = pd.Series(trial_labels_with_pattern).value_counts()

print("\n--- Analysis Complete ---")
print(f"Total MI validation trials analyzed: {total_mi_trials}")
print(f"Number of trials showing a contralateral pattern in at least one window: {num_trials_with_pattern}")
print(f"Percentage of trials following the pattern: {percentage_with_pattern:.2f}%\n")

print("Class distribution of trials that followed the pattern:")
if not class_distribution.empty:
    print(class_distribution.to_string())
else:
    print("No trials found with the specified pattern.")

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.signal import butter, lfilter

# --- 1. CONFIGURATION ---
# Kaggle environment path
BASE_PATH = '/kaggle/input/mtcaic3'

# EEG constants
SAMPLE_RATE = 250  # Hz
MU_BAND = (8, 12)  # Frequency band (in Hz) for checking contralateral activity

# Define all the time windows to be tested (in seconds)
TIME_WINDOWS = [
    (1.25, 2.25),
    (2.75, 3.75),
    (3.50, 4.50),
    (4.25, 5.25),
    (5.00, 6.00),
    (5.75, 6.75),
    (6.00, 7.00),
    (7.00, 8.00)
]

# --- 2. HELPER FUNCTIONS ---

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data, axis=0)
    return y

def load_trial_data(row, base_path, split='test'):
    """Loads raw EEG data for a single trial specified by a metadata row."""
    eeg_path = os.path.join(base_path, row['task'], split, row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
    eeg_data = pd.read_csv(eeg_path)
    
    trial_num = int(row['trial'])
    samples_per_trial = 2250  # MI: 9s * 250Hz
    
    start_idx = (trial_num - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial
    
    trial_data = eeg_data.iloc[start_idx:end_idx]
    return trial_data

def check_contralateral_pattern(trial_data_window, assumed_label):
    """
    Checks if a trial exhibits the expected contralateral ERD pattern for an ASSUMED label.
    """
    # Filter data for mu-band frequencies
    c3_filtered = bandpass_filter(trial_data_window['C3'], MU_BAND[0], MU_BAND[1], SAMPLE_RATE)
    c4_filtered = bandpass_filter(trial_data_window['C4'], MU_BAND[0], MU_BAND[1], SAMPLE_RATE)

    # Calculate power (variance)
    power_c3 = np.var(c3_filtered)
    power_c4 = np.var(c4_filtered)

    if assumed_label == 'Left':
        return power_c4 < power_c3
    elif assumed_label == 'Right':
        return power_c3 < power_c4
    return False

# --- 3. MAIN ANALYSIS SCRIPT ---

print("Loading and filtering test metadata for MI tasks...")
test_df = pd.read_csv(os.path.join(BASE_PATH, 'test.csv'))
mi_test_df = test_df[test_df['task'] == 'MI'].reset_index(drop=True)

trials_fitting_left = []
trials_fitting_right = []

print(f"\nAnalyzing {len(mi_test_df)} MI trials from the test set...")

# Iterate through each MI trial in the test set
for index, row in mi_test_df.iterrows():
    full_trial_data = load_trial_data(row, BASE_PATH)
    
    # --- Test 1: Assume the label is 'Left' ---
    found_left_pattern = False
    for start_t, end_t in TIME_WINDOWS:
        start_sample = int(start_t * SAMPLE_RATE)
        end_sample = int(end_t * SAMPLE_RATE)
        windowed_data = full_trial_data.iloc[start_sample:end_sample]
        
        if check_contralateral_pattern(windowed_data, 'Left'):
            trials_fitting_left.append(row['id'])
            found_left_pattern = True
            break # Pattern found, no need to check other windows for this assumption
            
    # --- Test 2: Assume the label is 'Right' ---
    found_right_pattern = False
    for start_t, end_t in TIME_WINDOWS:
        start_sample = int(start_t * SAMPLE_RATE)
        end_sample = int(end_t * SAMPLE_RATE)
        windowed_data = full_trial_data.iloc[start_sample:end_sample]

        if check_contralateral_pattern(windowed_data, 'Right'):
            trials_fitting_right.append(row['id'])
            found_right_pattern = True
            break # Pattern found, no need to check other windows for this assumption

# --- 4. REPORTING RESULTS ---

total_mi_trials = len(mi_test_df)
set_fit_left = set(trials_fitting_left)
set_fit_right = set(trials_fitting_right)

# Calculate counts for different scenarios
num_fit_left = len(set_fit_left)
num_fit_right = len(set_fit_right)
num_fit_both = len(set_fit_left.intersection(set_fit_right))
num_fit_only_left = num_fit_left - num_fit_both
num_fit_only_right = num_fit_right - num_fit_both
num_fit_neither = total_mi_trials - len(set_fit_left.union(set_fit_right))

print("\n--- Test Set Analysis Complete ---")
print(f"Total MI test trials analyzed: {total_mi_trials}\n")

# --- Report on 'Left' assumption ---
perc_left = (num_fit_left / total_mi_trials) * 100 if total_mi_trials > 0 else 0
print(f"Trials fitting 'Left' pattern: {num_fit_left} ({perc_left:.2f}%)")

# --- Report on 'Right' assumption ---
perc_right = (num_fit_right / total_mi_trials) * 100 if total_mi_trials > 0 else 0
print(f"Trials fitting 'Right' pattern: {num_fit_right} ({perc_right:.2f}%)\n")

# --- Summary Report ---
print("--- Summary of Pattern Fits ---")
print(f"Trials fitting ONLY the 'Left' pattern: \t{num_fit_only_left}")
print(f"Trials fitting ONLY the 'Right' pattern: \t{num_fit_only_right}")
print(f"Trials fitting BOTH patterns (ambiguous): \t{num_fit_both}")
print(f"Trials fitting NEITHER pattern: \t\t{num_fit_neither}")
print("-----------------------------------")
print(f"Total trials accounted for: {num_fit_only_left + num_fit_only_right + num_fit_both + num_fit_neither}")

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.signal import butter, lfilter

# --- 1. CONFIGURATION ---
# Kaggle environment path
BASE_PATH = '/kaggle/input/mtcaic3'

# EEG constants
SAMPLE_RATE = 250  # Hz
MU_BAND = (8, 12)  # Frequency band (in Hz) for checking activity

# Define all the time windows to be tested (in seconds)
TIME_WINDOWS = [
    (1.25, 2.25),
    (2.75, 3.75),
    (3.50, 4.50),
    (4.25, 5.25),
    (5.00, 6.00),
    (5.75, 6.75),
    (6.00, 7.00),
    (7.00, 8.00)
]

# --- 2. HELPER FUNCTIONS ---

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data, axis=0)
    return y

def load_trial_data(row, base_path, split):
    """Loads raw EEG data for a single trial specified by a metadata row."""
    eeg_path = os.path.join(base_path, row['task'], split, row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
    eeg_data = pd.read_csv(eeg_path)
    
    trial_num = int(row['trial'])
    samples_per_trial = 2250  # MI: 9s * 250Hz
    start_idx = (trial_num - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial
    
    return eeg_data.iloc[start_idx:end_idx]

def check_pattern_types(trial_data_window, true_label):
    """
    Checks if a trial window shows a classic contralateral or an inverted pattern.
    Returns two booleans: (is_classic, is_inverted)
    """
    c3_filtered = bandpass_filter(trial_data_window['C3'], MU_BAND[0], MU_BAND[1], SAMPLE_RATE)
    c4_filtered = bandpass_filter(trial_data_window['C4'], MU_BAND[0], MU_BAND[1], SAMPLE_RATE)
    power_c3 = np.var(c3_filtered)
    power_c4 = np.var(c4_filtered)
    
    is_classic, is_inverted = False, False
    
    if true_label == 'Left':
        if power_c4 < power_c3: # Expected pattern
            is_classic = True
        elif power_c3 < power_c4: # Inverted pattern
            is_inverted = True
    elif true_label == 'Right':
        if power_c3 < power_c4: # Expected pattern
            is_classic = True
        elif power_c4 < power_c3: # Inverted pattern
            is_inverted = True
            
    return is_classic, is_inverted

# --- 3. MAIN ANALYSIS FUNCTION ---

def analyze_dataset_patterns(df, split_name):
    """
    Analyzes a full dataset (train or validation) to categorize each trial's pattern.
    """
    print(f"\n--- Analyzing {split_name.upper()} Set ---")
    
    mi_df = df[df['task'] == 'MI'].reset_index(drop=True)
    total_mi_trials = len(mi_df)
    
    results = {
        'classic': [], 'inverted': [], 'ambiguous': [], 'neither': []
    }

    for index, row in mi_df.iterrows():
        full_trial_data = load_trial_data(row, BASE_PATH, split_name)
        
        found_classic = False
        found_inverted = False
        
        for start_t, end_t in TIME_WINDOWS:
            start_sample = int(start_t * SAMPLE_RATE)
            end_sample = int(end_t * SAMPLE_RATE)
            windowed_data = full_trial_data.iloc[start_sample:end_sample]
            
            is_classic, is_inverted = check_pattern_types(windowed_data, row['label'])
            
            if is_classic:
                found_classic = True
            if is_inverted:
                found_inverted = True

        # Categorize the trial based on its behavior across all windows
        if found_classic and found_inverted:
            results['ambiguous'].append(row['label'])
        elif found_inverted:
            results['inverted'].append(row['label'])
        elif found_classic:
            results['classic'].append(row['label'])
        else:
            results['neither'].append(row['label'])

    # --- 4. REPORTING RESULTS ---
    print(f"Total MI trials analyzed: {total_mi_trials}\n")

    for pattern_type, labels in results.items():
        count = len(labels)
        percentage = (count / total_mi_trials) * 100 if total_mi_trials > 0 else 0
        
        print(f"Category: '{pattern_type.capitalize()}'")
        print(f"  - Total Count: {count} ({percentage:.2f}%)")
        
        if count > 0:
            distribution = pd.Series(labels).value_counts()
            print("  - Class Distribution:")
            for label, num in distribution.items():
                print(f"      {label}: {num}")
        print("-" * 25)

# --- 5. EXECUTION ---
# Load data
train_df = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
validation_df = pd.read_csv(os.path.join(BASE_PATH, 'validation.csv'))

# Run analysis
analyze_dataset_patterns(validation_df, 'validation')
analyze_dataset_patterns(train_df, 'train')

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.signal import butter, lfilter
from scipy.stats import kurtosis, skew
from mne.decoding import CSP
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# --- 1. CONFIGURATION ---
# Paths and Constants
BASE_PATH = '/kaggle/input/mtcaic3'
SAMPLE_RATE = 250
ALL_EEG_CHANNELS = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']

# Feature Engineering Bands (Hz)
FREQ_BANDS = {
    "delta": (1, 4),
    "theta": (4, 8),
    "alpha": (8, 12), # Same as Mu
    "beta": (13, 30),
    "gamma": (30, 50)
}

# Model and Training Parameters
CSP_COMPONENTS = 6
EPOCHS = 150
BATCH_SIZE = 64

# --- 2. HELPER FUNCTIONS ---

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    # Apply filter to each channel
    return np.apply_along_axis(lambda x: lfilter(b, a, x), 0, data)

def load_trial_data(row, base_path, split):
    """Loads raw EEG data for a single trial."""
    path = os.path.join(base_path, row['task'], split, row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
    eeg_data = pd.read_csv(path)
    samples_per_trial = 2250 # MI: 9s * 250Hz
    start_idx = (row['trial'] - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial
    return eeg_data.iloc[start_idx:end_idx][ALL_EEG_CHANNELS]

def engineer_features_for_trial(trial_data):
    """Extracts a rich set of engineered features from a single trial."""
    features = []
    
    # 1. Band Power Features
    for band, (low, high) in FREQ_BANDS.items():
        # Filter data
        filtered_data = bandpass_filter(trial_data, low, high, SAMPLE_RATE)
        # Log-variance as power estimate
        band_power = np.log(np.var(filtered_data, axis=0))
        features.extend(band_power)
        
        # 2. Spatial Features (Asymmetry) for this band
        # C3 vs C4
        features.append(band_power[ALL_EEG_CHANNELS.index('C3')] - band_power[ALL_EEG_CHANNELS.index('C4')])
        # PO7 vs PO8
        features.append(band_power[ALL_EEG_CHANNELS.index('PO7')] - band_power[ALL_EEG_CHANNELS.index('PO8')])

    # 3. Time-Domain Statistical Features on raw data
    features.extend(kurtosis(trial_data, axis=0))
    features.extend(skew(trial_data, axis=0))
    
    return features

def create_dl_model(input_shape):
    """Creates a Deep Learning model for classification."""
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# --- 3. DATA PREPARATION ---

def prepare_dataset(df, split):
    """Loads all data, engineers features, and prepares it for the model."""
    print(f"Preparing {split} dataset...")
    mi_df = df[df['task'] == 'MI'].reset_index(drop=True)
    
    # --- Load all raw trials first ---
    raw_trials = [load_trial_data(row, BASE_PATH, split) for _, row in mi_df.iterrows()]
    raw_trials_np = np.array([trial.to_numpy() for trial in raw_trials]) # Shape: (n_trials, n_samples, n_channels)
    
    # --- A: Perform Feature Engineering ---
    print(f"  - Engineering features for {len(raw_trials_np)} trials...")
    engineered_features = np.array([engineer_features_for_trial(trial) for trial in raw_trials])
    
    # --- B: Perform CSP Feature Extraction ---
    # CSP needs data in shape (n_trials, n_channels, n_samples)
    csp_input = raw_trials_np.transpose(0, 2, 1)
    
    # Labels needed for fitting CSP
    labels = mi_df['label'].values if 'label' in mi_df.columns else None
    
    return engineered_features, csp_input, labels

# Load metadata
train_df = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
validation_df = pd.read_csv(os.path.join(BASE_PATH, 'validation.csv'))

# Prepare datasets
X_eng_train, X_csp_train_input, y_train_labels = prepare_dataset(train_df, 'train')
X_eng_val, X_csp_val_input, y_val_labels = prepare_dataset(validation_df, 'validation')

# --- 4. CSP FITTING AND TRANSFORMATION ---
print("\nFitting CSP on training data and transforming datasets...")
csp = CSP(n_components=CSP_COMPONENTS, reg=None, log=True, norm_trace=False)

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train_labels)
y_val_encoded = le.transform(y_val_labels)

# Fit CSP on training data and transform both sets
X_csp_train = csp.fit_transform(X_csp_train_input, y_train_encoded)
X_csp_val = csp.transform(X_csp_val_input)

# --- 5. COMBINE FEATURES AND SCALE ---
print("Combining engineered and CSP features...")
X_train_combined = np.concatenate((X_eng_train, X_csp_train), axis=1)
X_val_combined = np.concatenate((X_eng_val, X_csp_val), axis=1)

print(f"Final combined feature shape: {X_train_combined.shape}")

print("Scaling features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_combined)
X_val_scaled = scaler.transform(X_val_combined)

# --- 6. MODEL TRAINING ---
print("\nBuilding and training the Deep Learning model...")

# Calculate class weights for imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)
class_weights_dict = dict(enumerate(class_weights))
print(f"Using class weights: {class_weights_dict}")

model = create_dl_model(X_train_scaled.shape[1])
model.summary()

early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True, mode='max')

history = model.fit(
    X_train_scaled, y_train_encoded,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val_scaled, y_val_encoded),
    class_weight=class_weights_dict,
    callbacks=[early_stopping],
    verbose=2
)

# --- 7. EVALUATION ---
print("\n--- Final Model Evaluation on Validation Set ---")
loss, accuracy = model.evaluate(X_val_scaled, y_val_encoded, verbose=0)
print(f"\nValidation Accuracy: {accuracy:.4f}")
print(f"Validation Loss: {loss:.4f}")

# Generate predictions for detailed report
y_pred_probs = model.predict(X_val_scaled)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

# Classification Report
print("\nClassification Report:")
print(classification_report(y_val_encoded, y_pred, target_names=le.classes_))

# Confusion Matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_val_encoded, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix for Validation Data')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.signal import butter, lfilter
from scipy.stats import kurtosis, skew
from mne.decoding import CSP
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# --- 1. CONFIGURATION ---
BASE_PATH = '/kaggle/input/mtcaic3'
SAMPLE_RATE = 250
ALL_EEG_CHANNELS = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
FREQ_BANDS = {"delta": (1, 4), "theta": (4, 8), "alpha": (8, 12), "beta": (13, 30), "gamma": (30, 50)}
CSP_COMPONENTS = 6
EPOCHS = 150
BATCH_SIZE = 64

# --- 2. HELPER FUNCTIONS (Unchanged) ---
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return np.apply_along_axis(lambda x: lfilter(b, a, x), 0, data)

def load_trial_data(row, base_path, split):
    path = os.path.join(base_path, row['task'], split, row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
    eeg_data = pd.read_csv(path)
    samples_per_trial = 2250
    start_idx = (row['trial'] - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial
    return eeg_data.iloc[start_idx:end_idx][ALL_EEG_CHANNELS]

def engineer_features_for_trial(trial_data):
    features = []
    for band, (low, high) in FREQ_BANDS.items():
        filtered_data = bandpass_filter(trial_data, low, high, SAMPLE_RATE)
        band_power = np.log(np.var(filtered_data, axis=0))
        features.extend(band_power)
        features.append(band_power[ALL_EEG_CHANNELS.index('C3')] - band_power[ALL_EEG_CHANNELS.index('C4')])
        features.append(band_power[ALL_EEG_CHANNELS.index('PO7')] - band_power[ALL_EEG_CHANNELS.index('PO8')])
    features.extend(kurtosis(trial_data, axis=0))
    features.extend(skew(trial_data, axis=0))
    return features

def create_dl_model(input_shape):
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# --- 3. DATA PREPARATION ---

# Load metadata and filter for MI task
train_df = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
validation_df = pd.read_csv(os.path.join(BASE_PATH, 'validation.csv'))
mi_train_df = train_df[train_df['task'] == 'MI'].reset_index(drop=True)
mi_validation_df = validation_df[validation_df['task'] == 'MI'].reset_index(drop=True)

# --- NEW STEP: ONE-HOT ENCODE SUBJECT ID ---
print("One-hot encoding subject IDs...")
subject_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
# Fit on training subjects and transform both sets
subject_train_onehot = subject_encoder.fit_transform(mi_train_df[['subject_id']])
subject_val_onehot = subject_encoder.transform(mi_validation_df[['subject_id']])
print(f"Shape of one-hot encoded subject features (train): {subject_train_onehot.shape}")

def prepare_features(df, split):
    """Loads data and prepares engineered + CSP features."""
    print(f"Preparing features for {split} set...")
    raw_trials = [load_trial_data(row, BASE_PATH, split) for _, row in df.iterrows()]
    raw_trials_np = np.array([trial.to_numpy() for trial in raw_trials])
    
    # Engineered features
    engineered_features = np.array([engineer_features_for_trial(trial) for trial in raw_trials])
    
    # CSP input shape
    csp_input = raw_trials_np.transpose(0, 2, 1)
    
    labels = df['label'].values if 'label' in df.columns else None
    return engineered_features, csp_input, labels

# Prepare the signal-based features
X_eng_train, X_csp_train_input, y_train_labels = prepare_features(mi_train_df, 'train')
X_eng_val, X_csp_val_input, y_val_labels = prepare_features(mi_validation_df, 'validation')

# --- 4. CSP FITTING AND TRANSFORMATION ---
print("\nFitting CSP and transforming datasets...")
csp = CSP(n_components=CSP_COMPONENTS, reg=None, log=True, norm_trace=False)
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train_labels)
y_val_encoded = le.transform(y_val_labels)

X_csp_train = csp.fit_transform(X_csp_train_input, y_train_encoded)
X_csp_val = csp.transform(X_csp_val_input)

# --- 5. COMBINE ALL FEATURES (SIGNAL + SUBJECT) AND SCALE ---
print("Combining engineered, CSP, and subject ID features...")
X_train_combined = np.concatenate((X_eng_train, X_csp_train, subject_train_onehot), axis=1)
X_val_combined = np.concatenate((X_eng_val, X_csp_val, subject_val_onehot), axis=1)

print(f"Final combined feature shape (train): {X_train_combined.shape}")
print(f"Final combined feature shape (validation): {X_val_combined.shape}")

print("Scaling combined features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_combined)
X_val_scaled = scaler.transform(X_val_combined)

# --- 6. MODEL TRAINING ---
print("\nBuilding and training the Deep Learning model...")

class_weights = compute_class_weight('balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)
class_weights_dict = dict(enumerate(class_weights))
print(f"Using class weights: {class_weights_dict}")

# Create model with the new input shape
model = create_dl_model(X_train_scaled.shape[1])
model.summary()

early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True, mode='max')

history = model.fit(
    X_train_scaled, y_train_encoded,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val_scaled, y_val_encoded),
    class_weight=class_weights_dict,
    callbacks=[early_stopping],
    verbose=2
)

# --- 7. EVALUATION ---
print("\n--- Final Model Evaluation on Validation Set ---")
loss, accuracy = model.evaluate(X_val_scaled, y_val_encoded, verbose=0)
print(f"\nValidation Accuracy: {accuracy:.4f}")
print(f"Validation Loss: {loss:.4f}")

y_pred_probs = model.predict(X_val_scaled)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

print("\nClassification Report:")
print(classification_report(y_val_encoded, y_pred, target_names=le.classes_))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_val_encoded, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix for Validation Data')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.signal import butter, lfilter
from scipy.stats import kurtosis, skew
from mne.decoding import CSP
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, classification_report
import warnings

# --- 0. SETUP ---
# Suppress warnings and TensorFlow messages for cleaner output
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# --- 1. CONFIGURATION ---
BASE_PATH = '/kaggle/input/mtcaic3'
SAMPLE_RATE = 250
ALL_EEG_CHANNELS = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
FREQ_BANDS = {
    "delta": (1, 4),
    "theta": (4, 8),
    "alpha": (8, 12),
    "beta": (13, 30),
    "gamma": (30, 50)
}
CSP_COMPONENTS = 4 # Reduced components for smaller per-subject data
EPOCHS = 100
BATCH_SIZE = 8     # Smaller batch size for smaller data

# --- 2. HELPER FUNCTIONS ---
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return np.apply_along_axis(lambda x: lfilter(b, a, x), 0, data)

def load_trial_data(row, base_path, split):
    """Loads raw EEG data for a single trial."""
    path = os.path.join(base_path, row['task'], split, row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
    eeg_data = pd.read_csv(path)
    samples_per_trial = 2250
    start_idx = (row['trial'] - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial
    return eeg_data.iloc[start_idx:end_idx][ALL_EEG_CHANNELS]

def engineer_features_for_trial(trial_data):
    """Extracts a set of engineered features from a single trial."""
    features = []
    for band, (low, high) in FREQ_BANDS.items():
        filtered_data = bandpass_filter(trial_data, low, high, SAMPLE_RATE)
        band_power = np.log(np.var(filtered_data, axis=0))
        features.extend(band_power)
        # Add asymmetry feature for this band
        features.append(band_power[ALL_EEG_CHANNELS.index('C3')] - band_power[ALL_EEG_CHANNELS.index('C4')])
    
    # Add statistical features on raw data
    features.extend(kurtosis(trial_data, axis=0))
    return features

def create_dl_model(input_shape):
    """Creates a simple Deep Learning model for classification."""
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# --- 3. DATA PREPARATION ---
print("Loading and preparing metadata...")
train_df = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
mi_df = train_df[train_df['task'] == 'MI'].reset_index(drop=True)

# Identify all subjects in the training set
all_subjects = sorted(mi_df['subject_id'].unique())

# --- 4. SUBJECT-SPECIFIC TRAINING LOOP ---
subject_performance = {}

for subject_id in all_subjects:
    print(f"\n{'='*50}\nTraining model for Subject: {subject_id}\n{'='*50}")
    
    # --- A. Isolate data for the current subject ---
    subject_df = mi_df[mi_df['subject_id'] == subject_id]
    
    # --- B. Split subject's data by session ---
    train_sessions_df = subject_df[subject_df['trial_session'].isin([1, 2, 3, 4, 5, 6])]
    val_sessions_df = subject_df[subject_df['trial_session'] == 7]
    test_sessions_df = subject_df[subject_df['trial_session'] == 8]

    # Ensure all data splits are available
    if len(train_sessions_df) == 0 or len(val_sessions_df) == 0 or len(test_sessions_df) == 0:
        print(f"Skipping {subject_id} due to insufficient sessions for a full train/val/test split.")
        continue

    # --- C. Prepare Features for each data split ---
    le = LabelEncoder()
    y_train = le.fit_transform(train_sessions_df['label'])
    y_val = le.transform(val_sessions_df['label'])
    y_test = le.transform(test_sessions_df['label'])

    def prepare_features(df):
        # The 'train' directory contains all sessions for subjects S1-S30
        raw_trials = [load_trial_data(row, BASE_PATH, 'train') for _, row in df.iterrows()]
        raw_trials_np = np.array([trial.to_numpy() for trial in raw_trials])
        engineered = np.array([engineer_features_for_trial(trial) for trial in raw_trials])
        csp_input = raw_trials_np.transpose(0, 2, 1)
        return engineered, csp_input

    X_eng_train, X_csp_train_in = prepare_features(train_sessions_df)
    X_eng_val, X_csp_val_in = prepare_features(val_sessions_df)
    X_eng_test, X_csp_test_in = prepare_features(test_sessions_df)
    
    # --- D. Fit transformers ON SUBJECT'S TRAINING DATA ONLY ---
    csp = CSP(n_components=CSP_COMPONENTS, reg=None, log=True)
    X_csp_train = csp.fit_transform(X_csp_train_in, y_train)
    X_train_combined = np.concatenate((X_eng_train, X_csp_train), axis=1)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_combined)

    # --- E. Transform val and test sets with the FITTED transformers ---
    X_csp_val = csp.transform(X_csp_val_in)
    X_val_combined = np.concatenate((X_eng_val, X_csp_val), axis=1)
    X_val_scaled = scaler.transform(X_val_combined)
    
    X_csp_test = csp.transform(X_csp_test_in)
    X_test_combined = np.concatenate((X_eng_test, X_csp_test), axis=1)
    X_test_scaled = scaler.transform(X_test_combined)

    # --- F. Train the personalized model ---
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weights_dict = dict(enumerate(class_weights))
    
    model = create_dl_model(X_train_scaled.shape[1])
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True, mode='max', verbose=0)
    
    model.fit(X_train_scaled, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE,
              validation_data=(X_val_scaled, y_val),
              class_weight=class_weights_dict,
              callbacks=[early_stopping], verbose=0)
              
    # --- G. Evaluate on the subject's private test set (Session 8) ---
    y_pred_probs = model.predict(X_test_scaled, verbose=0)
    y_pred = (y_pred_probs > 0.5).astype(int).flatten()
    
    accuracy = accuracy_score(y_test, y_pred)
    subject_performance[subject_id] = accuracy
    
    print(f"--- Results for {subject_id} ---")
    print(f"Test Accuracy on Session 8: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0))

# --- 5. FINAL SUMMARY ---
print(f"\n\n{'='*50}\nOVERALL PERFORMANCE SUMMARY\n{'='*50}")
valid_accuracies = [acc for acc in subject_performance.values()]
if valid_accuracies:
    average_accuracy = np.mean(valid_accuracies)
    std_dev_accuracy = np.std(valid_accuracies)
    
    print(f"Average Test Accuracy across {len(valid_accuracies)} subjects: {average_accuracy:.4f}")
    print(f"Standard Deviation of Accuracy: {std_dev_accuracy:.4f}")
    print(f"Best performing subject accuracy: {np.max(valid_accuracies):.4f}")
    print(f"Worst performing subject accuracy: {np.min(valid_accuracies):.4f}")
else:
    print("No subjects were successfully trained and evaluated.")

In [1]:
pip install pyriemann

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.7/121.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 89.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.3.1 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.3 which is incompatible.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.3.1 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, b

In [4]:
!pip install --no-cache-dir numpy scipy scikit-learn pyriemann

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 172.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 178.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 216.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 157.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.7/121.7 kB 247.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.3.1 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.16.0 which is incompatible.
mkl-umath 0.1.1 requires numpy<1.27.0,

In [ ]:
# Robust EEG Classification Pipeline for MTC-AIC3 Competition
# Final Fixed Version

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, regularizers
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pywt
import warnings
warnings.filterwarnings('ignore')

# Set base path
base_path = "/kaggle/input/mtcaic3"
tf.random.set_seed(42)
np.random.seed(42)

# =====================================================================
### DATA LOADING & PREPROCESSING FUNCTIONS (FIXED) ###

def load_session_stats():
    """Precompute session statistics for adaptive normalization"""
    session_stats = {}
    for task in ['MI', 'SSVEP']:
        for dataset in ['train', 'validation', 'test']:
            task_path = os.path.join(base_path, task, dataset)
            subjects = [s for s in os.listdir(task_path) if s.startswith('S')]
            for subject in subjects:
                sessions = os.listdir(os.path.join(task_path, subject))
                for session in sessions:
                    path = os.path.join(task_path, subject, session, 'EEGdata.csv')
                    if os.path.exists(path):
                        try:
                            data = pd.read_csv(path).iloc[:, 1:9]  # EEG channels only
                            key = f"{task}_{dataset}_{subject}_{session}"
                            session_stats[key] = {
                                'mean': data.mean(axis=0).values,
                                'std': data.std(axis=0).values + 1e-8
                            }
                        except Exception as e:
                            print(f"Error processing {path}: {e}")
    return session_stats

def wavelet_denoise(signal, wavelet='db4', level=5):
    """Wavelet-based denoising for EEG signals"""
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    sigma = np.median(np.abs(coeffs[-level])) / 0.6745
    uthresh = sigma * np.sqrt(2 * np.log(len(signal)))
    coeffs[1:] = [pywt.threshold(c, value=uthresh, mode='soft') for c in coeffs[1:]]
    return pywt.waverec(coeffs, wavelet)

def extract_robust_features(eeg_data):
    """Feature engineering resistant to deceptive patterns"""
    # Channel indices: C3=1, C4=3, PO7=5, PO8=7 (0-indexed)
    c3 = eeg_data[:, 1]
    c4 = eeg_data[:, 3]
    po7 = eeg_data[:, 5]
    po8 = eeg_data[:, 7]
    
    # Asymmetry features with sign correction
    motor_asym = (np.mean(c3**2) - np.mean(c4**2)) * np.sign(np.mean(c3**2) + np.mean(c4**2))
    visual_asym = (np.mean(po7**2) - np.mean(po8**2)) * np.sign(np.mean(po7**2) + np.mean(po8**2))
    
    # Frequency band powers (Mu: 8-12Hz, Beta: 13-30Hz)
    freqs = np.fft.rfftfreq(len(c3), 1/250)
    fft_c3 = np.abs(np.fft.rfft(c3))
    mu_power = np.mean(fft_c3[(freqs >= 8) & (freqs <= 12)])
    beta_power = np.mean(fft_c3[(freqs >= 13) & (freqs <= 30)])
    
    return np.array([motor_asym, visual_asym, mu_power, beta_power])

def load_trial_data(row, session_stats):
    """Load and preprocess EEG data for a single trial"""
    # Access row attributes directly since we're using itertuples()
    id_num = row.id
    task = row.task
    subject_id = row.subject_id
    trial_session = row.trial_session
    trial_num = row.trial
    
    # Determine dataset based on ID
    dataset = 'train' if id_num <= 4800 else 'validation' if id_num <= 4900 else 'test'
    
    # Build EEG path
    eeg_path = os.path.join(
        base_path, 
        task, 
        dataset, 
        subject_id, 
        str(trial_session), 
        'EEGdata.csv'
    )
    
    # Load EEG data
    try:
        eeg_data = pd.read_csv(eeg_path).iloc[:, 1:9].values  # EEG channels only
    except Exception as e:
        print(f"Error loading {eeg_path}: {e}")
        return np.zeros((2250, 8)) if task == 'MI' else np.zeros((1750, 8)), np.zeros(4)
    
    # Calculate trial indices
    samples_per_trial = 2250 if task == 'MI' else 1750
    start_idx = (trial_num - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial
    trial_data = eeg_data[start_idx:end_idx]
    
    # Get session key for normalization
    session_key = f"{task}_{dataset}_{subject_id}_{trial_session}"
    stats = session_stats.get(session_key, None)
    
    # Adaptive normalization
    if stats:
        trial_data = (trial_data - stats['mean']) / stats['std']
    
    # Wavelet denoising per channel
    denoised_data = np.zeros_like(trial_data)
    for i in range(8):
        try:
            denoised_data[:, i] = wavelet_denoise(trial_data[:, i])
        except:
            denoised_data[:, i] = trial_data[:, i]  # Fallback to original
    
    # Feature extraction
    features = extract_robust_features(denoised_data)
    
    return denoised_data, features

def create_deceptive_sample(eeg_data):
    """Generate deceptive sample by flipping channels"""
    flipped = eeg_data.copy()
    # Swap C3<->C4 and PO7<->PO8
    flipped[:, [1, 3]] = flipped[:, [3, 1]]
    flipped[:, [5, 7]] = flipped[:, [7, 5]]
    return flipped

# =====================================================================
### DATA PREPARATION ###

# Load metadata
train_df = pd.read_csv(os.path.join(base_path, 'train.csv'))
val_df = pd.read_csv(os.path.join(base_path, 'validation.csv'))
test_df = pd.read_csv(os.path.join(base_path, 'test.csv'))

# Filter for Motor Imagery (MI) only
train_df = train_df[train_df['task'] == 'MI']
val_df = val_df[val_df['task'] == 'MI']
test_df = test_df[test_df['task'] == 'MI']

# Precompute session statistics
print("Precomputing session statistics...")
session_stats = load_session_stats()
print(f"Loaded stats for {len(session_stats)} sessions")

# Prepare data storage
X_train_raw, X_train_feat, y_train = [], [], []
X_val_raw, X_val_feat, y_val = [], [], []

# Label encoder
le = LabelEncoder()
le.fit(train_df['label'])

# Process training data
print("Processing training data...")
for i, row in enumerate(train_df.itertuples(index=False), 1):
    if i % 100 == 0:
        print(f"Processed {i}/{len(train_df)} training trials")
    eeg_data, features = load_trial_data(row, session_stats)
    label = le.transform([row.label])[0]
    
    # Add original sample
    X_train_raw.append(eeg_data)
    X_train_feat.append(features)
    y_train.append(label)
    
    # Add deceptive sample (same label)
    deceptive_eeg = create_deceptive_sample(eeg_data)
    deceptive_feat = extract_robust_features(deceptive_eeg)
    X_train_raw.append(deceptive_eeg)
    X_train_feat.append(deceptive_feat)
    y_train.append(label)

# Process validation data
print("Processing validation data...")
for i, row in enumerate(val_df.itertuples(index=False), 1):
    if i % 20 == 0 or i == len(val_df):
        print(f"Processed {i}/{len(val_df)} validation trials")
    eeg_data, features = load_trial_data(row, session_stats)
    label = le.transform([row.label])[0]
    X_val_raw.append(eeg_data)
    X_val_feat.append(features)
    y_val.append(label)

# Convert to arrays
X_train_raw = np.array(X_train_raw)
X_train_feat = np.array(X_train_feat)
y_train = np.array(y_train)
X_val_raw = np.array(X_val_raw)
X_val_feat = np.array(X_val_feat)
y_val = np.array(y_val)

print(f"Training data shapes - Raw: {X_train_raw.shape}, Features: {X_train_feat.shape}")
print(f"Validation data shapes - Raw: {X_val_raw.shape}, Features: {X_val_feat.shape}")

# Feature scaling
feat_scaler = StandardScaler()
X_train_feat = feat_scaler.fit_transform(X_train_feat)
X_val_feat = feat_scaler.transform(X_val_feat)

# =====================================================================
### SESSION-AWARE MODEL ARCHITECTURE ###

def build_session_aware_model(input_shape, num_features, num_sessions=8):
    """Hybrid model with session embedding for non-stationarity adaptation"""
    # Inputs
    eeg_input = layers.Input(shape=input_shape, name='eeg_input')
    feat_input = layers.Input(shape=(num_features,), name='feat_input')
    session_input = layers.Input(shape=(1,), dtype='int32', name='session_input')
    
    # Session embedding
    session_embed = layers.Embedding(
        input_dim=num_sessions+1, 
        output_dim=4, 
        embeddings_regularizer=regularizers.l2(1e-4)
    )(session_input)
    session_vec = layers.Flatten()(session_embed)
    
    # EEG processing branch
    x = layers.Conv1D(32, 3, padding='same', activation='elu')(eeg_input)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(32, 30, padding='same', activation='elu')(x)
    x = layers.SpatialDropout1D(0.4)(x)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Conv1D(64, 15, padding='same', activation='elu')(x)
    x = layers.SpatialDropout1D(0.4)(x)
    x = layers.GlobalAveragePooling1D()(x)
    
    # Feature processing branch
    y = layers.Dense(16, activation='elu')(feat_input)
    y = layers.Dropout(0.3)(y)
    
    # Combine with session information
    combined = layers.concatenate([x, y, session_vec])
    z = layers.Dense(64, activation='elu')(combined)
    z = layers.Dropout(0.4)(z)
    z = layers.Dense(32, activation='elu')(z)
    output = layers.Dense(2, activation='softmax')(z)
    
    return models.Model(
        inputs=[eeg_input, feat_input, session_input], 
        outputs=output,
        name='SessionAwareEEGModel'
    )

# Build model
print("Building model...")
model = build_session_aware_model(
    input_shape=(2250, 8), 
    num_features=X_train_feat.shape[1]
)
model.summary()

# Custom loss to focus on deceptive patterns
def deceptive_pattern_loss(y_true, y_pred):
    ce_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    return ce_loss  # Simplified for compatibility

model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Session IDs for training
train_sessions = np.concatenate([train_df['trial_session'].values, 
                                 train_df['trial_session'].values])
val_sessions = val_df['trial_session'].values

# =====================================================================
### TRAINING ###

# Callbacks
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=5, 
    min_lr=1e-6,
    verbose=1
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=15,
    restore_best_weights=True,
    mode='max',
    verbose=1
)

# Train the model
print("Training model...")
history = model.fit(
    x=[X_train_raw, X_train_feat, train_sessions],
    y=y_train,
    validation_data=([X_val_raw, X_val_feat, val_sessions], y_val),
    epochs=100,
    batch_size=32,
    callbacks=[lr_scheduler, early_stopping],
    verbose=1
)

# =====================================================================
### TEST SET PREDICTION & SUBMISSION ###

# Prepare test data
X_test_raw, X_test_feat, test_sessions, test_ids = [], [], [], []
print("Processing test data...")
for i, row in enumerate(test_df.itertuples(index=False), 1):
    if i % 20 == 0 or i == len(test_df):
        print(f"Processed {i}/{len(test_df)} test trials")
    eeg_data, features = load_trial_data(row, session_stats)
    X_test_raw.append(eeg_data)
    X_test_feat.append(features)
    test_sessions.append(row.trial_session)
    test_ids.append(row.id)

X_test_raw = np.array(X_test_raw)
X_test_feat = np.array(X_test_feat)
X_test_feat = feat_scaler.transform(X_test_feat)
test_sessions = np.array(test_sessions)

# Generate predictions
print("Generating predictions...")
test_probs = model.predict([X_test_raw, X_test_feat, test_sessions], batch_size=32, verbose=1)
test_preds = le.inverse_transform(np.argmax(test_probs, axis=1))

# Create submission
submission = pd.DataFrame({'id': test_ids, 'label': test_preds})
submission = submission.sort_values('id').reset_index(drop=True)
submission.to_csv('submission.csv', index=False)
print("Submission saved to submission.csv")
print("Number of predictions:", len(submission))
print("First 5 predictions:")
print(submission.head())

Precomputing session statistics...
Loaded stats for 500 sessions
Processing training data...
Processed 100/2400 training trials
Processed 200/2400 training trials
Processed 300/2400 training trials
Processed 400/2400 training trials
Processed 500/2400 training trials
Processed 600/2400 training trials
Processed 700/2400 training trials
Processed 800/2400 training trials
Processed 900/2400 training trials
Processed 1000/2400 training trials
Processed 1100/2400 training trials
Processed 1200/2400 training trials
Processed 1300/2400 training trials
Processed 1400/2400 training trials
Processed 1500/2400 training trials
Processed 1600/2400 training trials
Processed 1700/2400 training trials
Processed 1800/2400 training trials
Processed 1900/2400 training trials
Processed 2000/2400 training trials
Processed 2100/2400 training trials
Processed 2200/2400 training trials
Processed 2300/2400 training trials
